<a href="https://colab.research.google.com/github/AlexandreBourrieau/ML/blob/main/Carnets%20Jupyter/S%C3%A9ries%20temporelles/PredictionSeries.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Chargement des données

In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import pandas as pd

On télécharge un script depuis Github permettant de télécharger un fichier stocké sur GoogleDrive, puis on utilise ce script écrit en Python pour télécharger le fichier `bitcoin.zip`. Enfin, on décompresse les données pour obtenir le fichier `bitstampUSD_1-min_data_2012-01-01_to_2021-03-31.csv` :

In [ ]:
# Récupération des données au format .csv

!git clone https://github.com/chentinghao/download_google_drive.git
!python download_google_drive/download_gdrive.py "1FZsEdpBm-AQ2L9n_pMnm6336-O_IVo7z" "/content/bitcoin.zip"
!unzip bitcoin.zip

Charge la série sous Pandas et affiche les informations du fichier :

In [ ]:
# Création de la série sous Pandas
serie = pd.read_csv("bitstampUSD_1-min_data_2012-01-01_to_2021-03-31.csv")
serie

# Pré-traitement des données

**1. Recherche des erreurs dans les données**

On commence par vérifier qu'il ne manque pas de dates. Pour cela, on vérifie qu'il y a bien 60 secondes entre deux Timestamp. Si on trouve un décalage non cohérent, on enregistre les informations dans une liste.

In [ ]:
# Fonction permettant de vérifier si chaque intervalle est bien de 60s
def recherche_erreur(fenetre):
  if fenetre.values[1] - fenetre.values[0] != 60:
    Timestamp_Errors.append(fenetre.values)
  return 0

# Définit une liste pour sauvegarder le résultat des recherches
Timestamp_Errors = []

# Applique la fonction sur une fenêtre glissante des données
serie.Timestamp.rolling(2).apply(recherche_erreur)

On affiche les erreurs trouvées :

In [ ]:
# Affiche les informations sur les erreurs trouvées

for erreur in Timestamp_Errors:
  print (pd.to_datetime(Timestamp_Errors[0],unit="s"))
  print((Timestamp_Errors[0][1] - Timestamp_Errors[0][0])/60 - 1)

On observe qu'il manque des données entre le 5 janvier 2015 à 9:12:00 et le 9 janvier 2015 à 21:05:00, soit 6472 données.

Recherchons maintenant le nombre de données manquantes :

In [ ]:
# Affichage du nombre total de données manquantes

data_manquantes = sum(np.isnan(serie['Open']))
print ("Nombre de données manquantes : %s" %data_manquantes)

On a donc en tout : 6472 + 1243608 = 1250080 données manquantes.

**2. Identification des erreurs**

On convertit maintenant les `Timestamp` (mesure de temps exprimé en seconde écoulé depuis le 01/01/1970 - 00:00:00 UTC) en format plus standard :

In [ ]:
# Conversion des timestamp en date
serie.Timestamp = pd.to_datetime(serie['Timestamp'], unit="s")
serie

On demande maintenant à échantillonner les données sur 60 secondes :

In [ ]:
# Echantillonnage de la série sur 1min
serie_minute = serie.set_index('Timestamp').resample('60s').asfreq()

# Récupère le nombre de données sans valeurs numériques
data_manquantes = sum(np.isnan(serie_minute['Open']))

# Affiche le nombre de données manquantes et la série sur 1min 
print ("Nombre de données manquantes : %s" %data_manquantes)
serie_minute

On obtient en tout 4863849 données après échantillonnage, soit (4863849-4857377) =  6472 données supplémentaires. Ceci est cohérent avec ce qu'on avait trouvé avant. Il manque 1250080 données. 

**3. Correction des données**

Pour corriger les données, on va tout simplement utiliser la fonction [fillna](https://pandas.pydata.org/docs/reference/api/pandas.Series.fillna.html) de Pandas avec la fonctionnalité de type `backfill` :

In [ ]:
# Applique la fonction de remplissage automatique des données non numérique avec l'option backfill
serie_minute = serie_minute.interpolate(method="slinear")
serie_minute = serie_minute.fillna(method="backfill")

In [ ]:
# Récupère le nombre de données non numériques et affiche les informations

data_manquantes = sum(np.isnan(serie_minute['Open']))
print ("Nombre de données manquantes : %s" %data_manquantes)
serie_minute

**4. Affichage des données**

In [ ]:
# Affiche la série
plt.figure(figsize=(15,5))
plt.plot(serie_minute.index, serie_minute.Open)
plt.title("Evolution du prix du BTC")

# Préparation des données

Nous allons réaliser des modélisations sur la série journalière, et pour une période allant du 1er avril 2013 au 31 mars 2021.

**1. Création de la série horaire pour la modélisation globale**

On va réaliser des prédictions à l'aide d'une série à fréquence journalière. On commence par tenter d'estimer les données manquantes à l'aide d'une interpolation linéaire à l'aide de la fonction [interpolate](https://pandas.pydata.org/docs/reference/api/pandas.Series.interpolate.html#pandas.Series.interpolate) de Pandas, puis on complète avec la méthode `backfill` si nécessaire.

In [ ]:
# Echantillonne la série sur 1 heure
serie_heure = serie.set_index('Timestamp').resample('1H').asfreq()

# Remplissage des données non numériques par interpolation linéraire
serie_heure = serie_heure.interpolate(method="slinear")

# Remplissage des données non numériques restantes par backfill
serie_heure = serie_heure.fillna(method="backfill")

# Affiche les informations
data_manquantes = sum(np.isnan(serie_heure['Open']))
print ("Nombre de données manquantes : %s" %data_manquantes)
serie_heure

In [ ]:
# Affiche la série
plt.figure(figsize=(15,5))
plt.plot(serie_heure.index, serie_heure.Open)
plt.title("Evolution du prix du BTC")

On construit une nouvelle série avec les dates retenues pour le début et la fin :

In [ ]:
# Définition des dates de début et de fin

date_debut = "2013-04-01 00:00:00"
date_fin = "2021-03-31 00:00:00"

serie_etude = serie_heure.loc[date_debut:date_fin].copy()
serie_etude

In [ ]:
serie_etude['x'] = np.linspace(0,serie_etude.index.size-1,serie_etude.index.size)
serie_etude

**2. Détection des anomalies dans la série "horaire"**

Les anomalies sont fréquentes dans les séries temporelles, et la performance des prédictions est souvent améliorée lorsque ces anomalies sont traitées.  
Pour avoir un apperçu de ces éventuelles anomalies, nous allons utiliser la méthode ["Isolation Forest"](https://scikit-learn.org/stable/modules/outlier_detection.html#isolation-forest) disponnible dans Scikit-learn.  

Les paramètres utilisés sont les suivants :
 - **n_estimators** : C'est le nombre de sous-groupes d'échantillons à utiliser. Une valeur de 128 ou 256 est préconnisée dans le document de recherche.
 - **max_samples** : C'est le nombre d'échantillons maximum à utiliser. Nous utiliserons l'ensemble des échantillons.
 - **max_features** :  C'est le nombre de motifs aléatoirement choisis sur chaque noeud de l'arbre. Nous choisirons un seul motif.
 - **contamination** : C'est le pourcentage estimé d'anomalies dans les données. Ce paramètre permet de régler la sensibilité de l'algorithme. On va commencer avec 5% et affiner si nécessaire par la suite.

In [ ]:
# Initialise le modèle
from sklearn.ensemble import IsolationForest

clf = IsolationForest(n_estimators=256,max_samples=serie_etude['Open'].size, contamination=0.05,max_features=1, verbose=1)
clf.fit(serie_etude['Open'].values.reshape(-1,1))

In [ ]:
# Réalise les prédictions
pred = clf.predict(serie_etude['Open'].values.reshape(-1,1))
pred

On ajoute maintenant ces informations dans la série journalière et on affiche les informations :

In [ ]:
# Ajoute une colonne "Anomalie" dans la série
serie_etude['Anomalies']=pred
serie_etude['Anomalies'] = serie_etude['Anomalies'].apply(lambda x: 1 if (x==-1) else 0)
serie_etude

In [ ]:
# Affiche les informations sur les anomalies
print(serie_etude['Anomalies'].value_counts())

**3. Affichage des anomalies sur le graphique**

In [ ]:
# Affiche la série

fig = px.line(x=serie_etude.index,y=serie_etude['Open'],title="Evolution du prix du BTC")
fig.add_trace(px.scatter(x=serie_etude.index,y=serie_etude['Anomalies']*serie_etude['Open'],color=serie_etude['Anomalies'].astype(np.bool)).data[0])

fig.update_xaxes(rangeslider_visible=True)
yaxis=dict(autorange = True,fixedrange= False)
fig.update_yaxes(yaxis)
fig.show()

Comme les anomalies détectées ne sembles pas cohérentes, nous n'allons pas les traiter...

# Analyse de la série

In [ ]:
serie_etude

**1. ACF & PACF**

In [ ]:
# ACF & PACF du bruit blanc

serie = serie_etude['Open']

from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

f1, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
f1.subplots_adjust(hspace=0.3,wspace=0.2)

plot_acf(serie, ax=ax1, lags = range(0,50))
ax1.set_title("Autocorrélation du bruit blanc")

plot_pacf(serie, ax=ax2, lags = range(0, 50))
ax2.set_title("Autocorrélation partielle du bruit blanc")

**2. Test de Dickey-Fuller**

In [ ]:
import statsmodels.api as sm

serie_test = serie

adf, p, usedlag, nobs, cvs, aic = sm.tsa.stattools.adfuller(serie_test)

adf_results_string = 'ADF: {}\np-value: {},\nN: {}, \ncritical values: {}'
print(adf_results_string.format(adf, p, nobs, cvs))

**3. Suppression de la tendance non linéaire et test de sationnarité**

In [ ]:
from scipy.stats import boxcox

serie_log, lam = boxcox(serie)

f1, (ax1,ax2) = plt.subplots(2, 1, figsize=(15, 5))
ax1.plot(serie_etude.index,serie_log)
ax2.plot(serie_etude.index,serie)

In [ ]:
import statsmodels.api as sm

serie_test = serie_log

adf, p, usedlag, nobs, cvs, aic = sm.tsa.stattools.adfuller(serie_test)

adf_results_string = 'ADF: {}\np-value: {},\nN: {}, \ncritical values: {}'
print(adf_results_string.format(adf, p, nobs, cvs))

***4. Suppression de la tendance linéaire et test de stationnarité***

In [ ]:
f1, (ax1, ax2) = plt.subplots(2, 1, figsize=(15, 5))

# Calcul des coefficients
x = np.linspace(0,len(serie_log),len(serie_log))
coefs = np.polyfit(x,serie_log,1)

# Calcul de la tendance non linéaire
trend = coefs[0]*np.power(x,1) + coefs[1]

# Calcul de la série sans tendance
serie_log_detrend = serie_log - trend

# Affiche les résultats
ax1.plot(trend)
ax1.plot(serie_log)
ax1.set_title("Série originale et tendance non linéaire")

ax2.plot(serie_log_detrend)
ax2.set_title("Série avec tendance non linéaire supprimée")

In [ ]:
# ACF & PACF du bruit blanc

from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

f1, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
f1.subplots_adjust(hspace=0.3,wspace=0.2)

plot_acf(serie_log_detrend, ax=ax1, lags = range(0,50))
ax1.set_title("Autocorrélation du bruit blanc")

plot_pacf(serie_log_detrend, ax=ax2, lags = range(0, 50))
ax2.set_title("Autocorrélation partielle du bruit blanc")

In [ ]:
import statsmodels.api as sm

serie_test = serie_log_detrend

adf, p, usedlag, nobs, cvs, aic = sm.tsa.stattools.adfuller(serie_test)

adf_results_string = 'ADF: {}\np-value: {},\nN: {}, \ncritical values: {}'
print(adf_results_string.format(adf, p, nobs, cvs))

**5. Différentiation**

In [ ]:
# Différenciation d'odre 1 et saisonnale à l'odre 1 et de période 12

from statsmodels.tsa.statespace.tools import diff

serie_log_detrend_diff1 = diff(serie_log_detrend,1)       # diff=1 ; diff_saison=1 ; periode = 12

plt.figure(figsize=(10, 6))
plt.plot(serie_log_detrend_diff1)
plt.title("Signal différencié d'ordre 1 + saisonalité")
plt.show()

In [ ]:
import statsmodels.api as sm

serie_test = serie_log_detrend_diff1

adf, p, usedlag, nobs, cvs, aic = sm.tsa.stattools.adfuller(serie_test)

adf_results_string = 'ADF: {}\np-value: {},\nN: {}, \ncritical values: {}'
print(adf_results_string.format(adf, p, nobs, cvs))

In [ ]:
# ACF & PACF du bruit blanc

from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

f1, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
f1.subplots_adjust(hspace=0.3,wspace=0.2)

plot_acf(serie_log_detrend_diff1, ax=ax1, lags = range(0,50))
ax1.set_title("Autocorrélation du bruit blanc")

plot_pacf(serie_log_detrend_diff1, ax=ax2, lags = range(0, 50))
ax2.set_title("Autocorrélation partielle du bruit blanc")

**5. Enregistrement des données dans le dataframe**

In [ ]:
serie_log_detrend_diff1 = np.insert(serie_log_detrend_diff1,0,0)

In [ ]:
serie_etude['diff'] = serie_log_detrend_diff1
serie_etude['diff'][0] = "Nan"
serie_etude

# Prépartion des datasets diff

In [ ]:
serie_etude

**1. Séparation des données en données pour l'entrainement et la validation**

On réserve 20% des données pour l'entrainement et le reste pour la validation :

In [ ]:
# Sépare les données en entrainement et tests
pourcentage = 0.8
temps_separation = int(len(serie_etude) * pourcentage)
date_separation = serie_etude.index[temps_separation]

serie_entrainement = serie_etude['diff'].iloc[1:temps_separation]
serie_test = serie_etude['diff'].iloc[temps_separation:]

print("Taille de l'entrainement : %d" %len(serie_entrainement))
print("Taille de la validation : %d" %len(serie_test))

On normalise les données :

In [ ]:
# Calcul de la moyenne et de l'écart type de la série
mean = tf.math.reduce_mean(np.asarray(serie_entrainement))
std = tf.math.reduce_std(np.asarray((serie_entrainement)))

# Normalisation des données
serie_entrainement = (serie_entrainement-mean)/std
serie_test = (serie_test-mean)/std

In [ ]:
# Affiche la série
fig, ax = plt.subplots(constrained_layout=True, figsize=(15,5))
ax.plot(serie_entrainement, label="Entrainement")
ax.plot(serie_test,label="Validation")

ax.set_title("Evolution du prix du BTC")

ax.legend()
plt.show()

**2. Création des datasets**

In [ ]:
# Fonction permettant de créer un dataset à partir des données de la série temporelle

def prepare_dataset_XY(serie, taille_fenetre, horizon, batch_size):
  dataset = tf.data.Dataset.from_tensor_slices(serie)
  dataset = dataset.window(taille_fenetre+horizon, shift=1, drop_remainder=True)
  dataset = dataset.flat_map(lambda x: x.batch(taille_fenetre + horizon))
  dataset = dataset.map(lambda x: (tf.expand_dims(x[0:taille_fenetre],axis=1),x[-1:]))
  dataset = dataset.batch(batch_size,drop_remainder=True).prefetch(1)
  return dataset

In [ ]:
# Définition des caractéristiques du dataset que l'on souhaite créer
taille_fenetre = 500
horizon = 1
batch_size = 100

# Création du dataset
dataset = prepare_dataset_XY(serie_entrainement,taille_fenetre,horizon,batch_size)
dataset_val = prepare_dataset_XY(serie_test,taille_fenetre,horizon,batch_size)

In [ ]:
print(len(list(dataset.as_numpy_iterator())))
for element in dataset.take(1):
  print(element[0].shape)
  print(element[1].shape)

On extrait maintenant les deux tenseurs (X,Y) pour l'entrainement :

In [ ]:
# Extrait les X,Y du dataset
#56x((1000,4,1),(1000,1)) => (56*1000,4,1) ; (56*1000,1)

x,y = tuple(zip(*dataset))

# Recombine les données
# (56,1000,4,1) => (56*128,4,1)
# (56,1000,1) => (56*128,1)
x_train = np.asarray(tf.reshape(np.asarray(x,dtype=np.float32),shape=(np.asarray(x).shape[0]*np.asarray(x).shape[1],taille_fenetre,1)))
y_train = np.asarray(tf.reshape(np.asarray(y,dtype=np.float32),shape=(np.asarray(y).shape[0]*np.asarray(y).shape[1])))

# Affiche les formats
print(x_train.shape)
print(y_train.shape)

Puis la même chose pour les données de validation :

In [ ]:
# Extrait les X,Y du dataset_val

x,y = tuple(zip(*dataset_val))

# Recombine les données

x_val = np.asarray(tf.reshape(np.asarray(x,dtype=np.float32),shape=(np.asarray(x).shape[0]*np.asarray(x).shape[1],taille_fenetre,1)))
y_val = np.asarray(tf.reshape(np.asarray(y,dtype=np.float32),shape=(np.asarray(y).shape[0]*np.asarray(y).shape[1])))

# Affiche les formats
print(x_val.shape)
print(y_val.shape)

# Prépartion des datasets

**1. Séparation des données en données pour l'entrainement et la validation**

On réserve 20% des données pour l'entrainement et le reste pour la validation :

In [ ]:
# Sépare les données en entrainement et tests
pourcentage = 0.8
temps_separation = int(len(serie_etude) * pourcentage)
date_separation = serie_etude.index[temps_separation]

serie_entrainement = serie_etude['Open'].iloc[:temps_separation]
serie_test = serie_etude['Open'].iloc[temps_separation:]

print("Taille de l'entrainement : %d" %len(serie_entrainement))
print("Taille de la validation : %d" %len(serie_test))

On normalise les données :

In [ ]:
# Calcul de la moyenne et de l'écart type de la série
mean = tf.math.reduce_mean(np.asarray(serie_entrainement))
std = tf.math.reduce_std(np.asarray((serie_entrainement)))

# Normalisation des données
serie_entrainement = (serie_entrainement-mean)/std
serie_test = (serie_test-mean)/std

In [ ]:
# Affiche la série
fig, ax = plt.subplots(constrained_layout=True, figsize=(15,5))
ax.plot(serie_entrainement, label="Entrainement")
ax.plot(serie_test,label="Validation")

ax.set_title("Evolution du prix du BTC")

ax.legend()
plt.show()

**2. Création des datasets**

In [ ]:
# Fonction permettant de créer un dataset à partir des données de la série temporelle

def prepare_dataset_XY(serie, taille_fenetre, horizon, batch_size):
  dataset = tf.data.Dataset.from_tensor_slices(serie)
  dataset = dataset.window(taille_fenetre+horizon, shift=1, drop_remainder=True)
  dataset = dataset.flat_map(lambda x: x.batch(taille_fenetre + horizon))
  dataset = dataset.map(lambda x: (tf.expand_dims(x[0:taille_fenetre],axis=1),x[-1:]))
  dataset = dataset.batch(batch_size,drop_remainder=True).prefetch(1)
  return dataset

In [ ]:
# Définition des caractéristiques du dataset que l'on souhaite créer
taille_fenetre = 500
horizon = 1
batch_size = 1000

# Création du dataset
dataset = prepare_dataset_XY(serie_entrainement,taille_fenetre,horizon,batch_size)
dataset_val = prepare_dataset_XY(serie_test,taille_fenetre,horizon,batch_size)

In [ ]:
print(len(list(dataset.as_numpy_iterator())))
for element in dataset.take(1):
  print(element[0].shape)
  print(element[1].shape)

On extrait maintenant les deux tenseurs (X,Y) pour l'entrainement :

In [ ]:
# Extrait les X,Y du dataset
#56x((1000,4,1),(1000,1)) => (56*1000,4,1) ; (56*1000,1)

x,y = tuple(zip(*dataset))

# Recombine les données
# (56,1000,4,1) => (56*128,4,1)
# (56,1000,1) => (56*128,1)
x_train = np.asarray(tf.reshape(np.asarray(x,dtype=np.float32),shape=(np.asarray(x).shape[0]*np.asarray(x).shape[1],taille_fenetre,1)))
y_train = np.asarray(tf.reshape(np.asarray(y,dtype=np.float32),shape=(np.asarray(y).shape[0]*np.asarray(y).shape[1])))

# Affiche les formats
print(x_train.shape)
print(y_train.shape)

Puis la même chose pour les données de validation :

In [ ]:
# Extrait les X,Y du dataset_val

x,y = tuple(zip(*dataset_val))

# Recombine les données

x_val = np.asarray(tf.reshape(np.asarray(x,dtype=np.float32),shape=(np.asarray(x).shape[0]*np.asarray(x).shape[1],taille_fenetre,1)))
y_val = np.asarray(tf.reshape(np.asarray(y,dtype=np.float32),shape=(np.asarray(y).shape[0]*np.asarray(y).shape[1])))

# Affiche les formats
print(x_val.shape)
print(y_val.shape)

# Prépartion des datasets X/Y(log)

**1. Séparation des données en données pour l'entrainement et la validation**

On réserve 20% des données pour l'entrainement et le reste pour la validation :

In [ ]:
# Sépare les données en entrainement et tests
pourcentage = 0.8
temps_separation = int(len(serie_etude) * pourcentage)
date_separation = serie_etude.index[temps_separation]

serie_entrainement_x = serie_etude['Open'].iloc[:temps_separation]
serie_entrainement_y = serie_etude['Open_log'].iloc[:temps_separation]

serie_test_x = serie_etude['Open'].iloc[temps_separation:]
serie_test_y = serie_etude['Open_log'].iloc[temps_separation:]

print("Taille de l'entrainement : %d" %len(serie_entrainement_x))
print("Taille de la validation : %d" %len(serie_test_x))

On normalise les labels :

In [ ]:
# Calcul de la moyenne et de l'écart type de la série
mean = tf.math.reduce_mean(np.asarray(serie_entrainement_y))
std = tf.math.reduce_std(np.asarray((serie_entrainement_y)))

# Normalisation des données
serie_entrainement_y = (serie_entrainement_y-mean)/std
serie_test_y = (serie_test_y-mean)/std

In [ ]:
# Affiche la série
f1, (ax1, ax2) = plt.subplots(2, 1, figsize=(15, 5))

ax1.plot(serie_entrainement_x, label="Entrainement")
ax1.plot(serie_test_x,label="Validation")

ax2.plot(serie_entrainement_y, label="Entrainement (log)")
ax2.plot(serie_test_y,label="Validation (log)")

plt.show()

**2. Création des datasets**

In [ ]:
def prepare_dataset_XY(serie_x,serie_y, taille_fenetre, horizon, batch_size):
  dataset_x = tf.data.Dataset.from_tensor_slices(serie_x)
  dataset_x = dataset_x.window(taille_fenetre, shift=1, drop_remainder=True)
  dataset_x = dataset_x.flat_map(lambda x: x.batch(taille_fenetre))
  dataset_x = dataset_x.map(lambda x: tf.expand_dims(x[0:taille_fenetre],axis=1))

  dataset_y = tf.data.Dataset.from_tensor_slices(serie_y)
  dataset_y = dataset_y.window(taille_fenetre+horizon, shift=1, drop_remainder=True)
  dataset_y = dataset_y.flat_map(lambda x: x.batch(taille_fenetre+horizon))
  dataset_y = dataset_y.map(lambda x: (x[-1:]))

  dataset = tf.data.Dataset.zip((dataset_x,dataset_y))
  dataset = dataset.batch(batch_size,drop_remainder=True).prefetch(1)
  return dataset

In [ ]:
  dataset = dataset.map(lambda x: (tf.expand_dims(x[0:taille_fenetre],axis=1),x[-1:]))

In [ ]:
x = np.linspace(0,1000,1001)
y = np.linspace(1001,2000,1000)

In [ ]:
toto = prepare_dataset_XY(x,x,10,1,5)

In [ ]:
print(len(list(toto.as_numpy_iterator())))
for element in toto.take(1):
  print(element)

In [ ]:
# Définition des caractéristiques du dataset que l'on souhaite créer
taille_fenetre = 50
horizon = 1
batch_size = 1000

# Création du dataset
dataset = prepare_dataset_XY(serie_entrainement_x,serie_entrainement_y,taille_fenetre,horizon,batch_size)
dataset_val = prepare_dataset_XY(serie_test_x,serie_test_y,taille_fenetre,horizon,batch_size)

In [ ]:
print(len(list(dataset.as_numpy_iterator())))
for element in dataset.take(1):
  print(element[0].shape)
  print(element[1].shape)

On extrait maintenant les deux tenseurs (X,Y) pour l'entrainement :

In [ ]:
# Extrait les X,Y du dataset
#56x((1000,50,1),(1000,1)) => (56*1000,50,1) ; (56*1000,1)

x,y = tuple(zip(*dataset))

# Recombine les données
# (56,1000,50,1) => (56*1000,50,1)
# (56,1000,1) => (56*1000,1)
x_train = np.asarray(tf.reshape(np.asarray(x,dtype=np.float32),shape=(np.asarray(x).shape[0]*np.asarray(x).shape[1],taille_fenetre,1)))
y_train = np.asarray(tf.reshape(np.asarray(y,dtype=np.float32),shape=(np.asarray(y).shape[0]*np.asarray(y).shape[1])))

# Affiche les formats
print(x_train.shape)
print(y_train.shape)

Puis la même chose pour les données de validation :

In [ ]:
# Extrait les X,Y du dataset_val

x,y = tuple(zip(*dataset_val))

# Recombine les données

x_val = np.asarray(tf.reshape(np.asarray(x,dtype=np.float32),shape=(np.asarray(x).shape[0]*np.asarray(x).shape[1],taille_fenetre,1)))
y_val = np.asarray(tf.reshape(np.asarray(y,dtype=np.float32),shape=(np.asarray(y).shape[0]*np.asarray(y).shape[1])))

# Affiche les formats
print(x_val.shape)
print(y_val.shape)

# Optimisation des hyperparamètres

**1. Création de la série horaire pour l'optimisation des hyperparamètres**

In [ ]:
# Définition des dates de début et de fin

date_debut = "2020-01-01 00:00:00"
date_fin = "2021-03-31 00:00:00"

serie_opti = serie_etude['Open'].loc[date_debut:date_fin].copy()
serie_opti

In [ ]:
# Affiche la série
plt.figure(figsize=(15,5))
plt.plot(serie_opti)
plt.title("Evolution du prix du BTC")

**1. Préparation des données**

On normalise les données :

In [ ]:
# Calcul de la moyenne et de l'écart type de la série
mean = tf.math.reduce_mean(np.asarray(serie_opti))
std = tf.math.reduce_std(np.asarray((serie_opti)))

# Normalisation des données
serie_opti = (serie_opti-mean)/std

**2. Création du dataset**

In [ ]:
# Fonction permettant de créer un dataset à partir des données de la série temporelle

def prepare_dataset_XY(serie, taille_fenetre, horizon, batch_size):
  dataset = tf.data.Dataset.from_tensor_slices(serie)
  dataset = dataset.window(taille_fenetre+horizon, shift=1, drop_remainder=True)
  dataset = dataset.flat_map(lambda x: x.batch(taille_fenetre + horizon))
  dataset = dataset.map(lambda x: (tf.expand_dims(x[0:taille_fenetre],axis=1),x[-1:]))
  dataset = dataset.batch(batch_size,drop_remainder=True).prefetch(1)
  return dataset

# Définition des caractéristiques du dataset que l'on souhaite créer
taille_fenetre = 4
horizon = 1
batch_size = 32

# Création du dataset
dataset = prepare_dataset_XY(serie_entrainement,taille_fenetre,horizon,batch_size)

In [ ]:
print(len(list(dataset.as_numpy_iterator())))
for element in dataset.take(1):
  print(element[0].shape)
  print(element[1].shape)

On extriat maintenant les données X et les labels Y du dataset :

In [ ]:
# Extrait les X,Y du dataset
# 272x((32,4,1),(32,1)) => (272*32,4,1) ; (272*32,1)

x,y = tuple(zip(*dataset))

# Recombine les données
# (272,32,4,1) => (272*32,4,1)
# (272,32,1) => (272*32,1)
x_train = np.asarray(tf.reshape(np.asarray(x,dtype=np.float32),shape=(np.asarray(x).shape[0]*np.asarray(x).shape[1],taille_fenetre,1)))
y_train = np.asarray(tf.reshape(np.asarray(y,dtype=np.float32),shape=(np.asarray(y).shape[0]*np.asarray(y).shape[1],1)))

# Affiche les formats
print(x_train.shape)
print(y_train.shape)

**3. Définition du modèle**

Dans le modèle, les paramètres dim_LSTM, l1_reg, l2_reg seront optimisés :

In [ ]:
def ModelLSTM(dim_LSTM = 10, l1_reg=0, l2_reg=0):

  entrees = tf.keras.layers.Input(shape=(taille_fenetre,1))

  # Encodeur
  s_encodeur = tf.keras.layers.LSTM(dim_LSTM, kernel_regularizer=tf.keras.regularizers.l1_l2(l1=l1_reg,l2=l2_reg))(entrees)
  
  # Générateur
  sortie = tf.keras.layers.Dense(1,kernel_regularizer=tf.keras.regularizers.l1_l2(l1=l1_reg,l2=l2_reg))(s_encodeur)

  # Construction du modèle
  model = tf.keras.Model(entrees,sortie)
  model.compile(loss='mse', optimizer='adam')
  return(model)

**4. Cross-validation**

In [ ]:
from keras.callbacks import EarlyStopping
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import KFold, TimeSeriesSplit, GridSearchCV

# Définitions des paramètres
dim_LSTM = [5,10,15,20,30,40]
l1_reg = [0,0.001,0.01,0.1]
l2_reg = [0,0.001,0.01,0.1]
batch_size = [32]

param_grid = {'dim_LSTM': dim_LSTM, 'l1_reg': l1_reg, 'l2_reg': l2_reg, 'batch_size': batch_size}
param_grid = {'dim_LSTM': dim_LSTM, 'batch_size': batch_size}

max_periodes = 5

# Surveillance de l'entrainement
es = EarlyStopping(monitor='loss', mode='min', verbose=1, patience=50, min_delta=1e-7, restore_best_weights=True)


tscv = TimeSeriesSplit(n_splits = 5)
model = KerasRegressor(build_fn=ModelLSTM, epochs=max_periodes, verbose=2)
grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=tscv, n_jobs=-1, verbose=3)

grid_result = grid.fit(x_train, y_train,callbacks=[es])

In [ ]:
# Affiche les résultats
print("Meilleur résultat : %f avec %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params_ = grid_result.cv_results_['params']
for mean, stdev, param_ in zip(means, stds, params_):
  print("%f (%f) with %r" % (mean, stdev, param_))

# Création du modèle LSTM de type encodeur-décodeur

**1. Création du réseau**

Par défaut, la dimension des vecteurs cachés est de 10 et aucune régularisation n'est utilisée.

In [ ]:
dim_LSTM = 100
l1_reg = 0.0
l2_reg = 0.0

# Définition de l'entrée du modèle
entrees = tf.keras.layers.Input(shape=(taille_fenetre,1),batch_size=batch_size)

# Encodeur
s_encodeur = tf.keras.layers.LSTM(dim_LSTM, kernel_regularizer=tf.keras.regularizers.l1_l2(l1=l1_reg,l2=l2_reg),stateful=True)(entrees)

# Décodeur
s_decodeur = tf.keras.layers.Dense(dim_LSTM,activation="tanh",kernel_regularizer=tf.keras.regularizers.l1_l2(l1=l1_reg,l2=l2_reg))(s_encodeur)
s_decodeur = tf.keras.layers.Concatenate()([s_decodeur,s_encodeur])

# Générateur
sortie = tf.keras.layers.Dense(1,kernel_regularizer=tf.keras.regularizers.l1_l2(l1=l1_reg,l2=l2_reg))(s_decodeur)

# Construction du modèle
model = tf.keras.Model(entrees,sortie)
model.summary()

In [ ]:
dim_LSTM = 100
l1_reg = 0.0
l2_reg = 0.0


class PenalizedLoss(tf.keras.losses.Loss):
  def __init__(self,reduction=tf.keras.losses.Reduction.AUTO,name='Penalized_loss'):
    self.reduction = reduction
    self.name = name
    self.n = 0

  def __call__(self, y_true, y_pred, sample_weight=None):
    print(y_true.shape)
    print(y_pred.shape)
    if self.n == 0:
      self.n = 1
      error = (tf.keras.metrics.mse(y_true,y_pred))
      return (error)
    else:
      error = (tf.keras.metrics.mse(y_true,y_pred))
      return (error)

def make_model():
  # loss
  loss = PenalizedLoss()

  # Définition de l'entrée du modèle
  entrees = tf.keras.layers.Input(shape=(taille_fenetre,1))

  # Encodeur
  s_encodeur = tf.keras.layers.LSTM(dim_LSTM, kernel_regularizer=tf.keras.regularizers.l1_l2(l1=l1_reg,l2=l2_reg))(entrees)

  # Décodeur
  s_decodeur = tf.keras.layers.Dense(dim_LSTM,activation="tanh",kernel_regularizer=tf.keras.regularizers.l1_l2(l1=l1_reg,l2=l2_reg))(s_encodeur)
  s_decodeur = tf.keras.layers.Concatenate()([s_decodeur,s_encodeur])

  # Générateur
  sortie = tf.keras.layers.Dense(1,kernel_regularizer=tf.keras.regularizers.l1_l2(l1=l1_reg,l2=l2_reg))(s_decodeur)

  # Construction du modèle
  model = tf.keras.Model(entrees,sortie)
  optimiseur=tf.keras.optimizers.Adam()
  model.compile(loss=loss, optimizer=optimiseur)
  return model

model = make_model()
model.summary()

**2. Optimisation de l'apprentissage**

Pour accélérer le traitement des données, nous n'allons pas utiliser l'intégralité des données pendant la mise à jour du gradient, comme cela a été fait jusqu'à présent (en utilisant le dataset).  
Cette fois-ci, nous allons forcer les mises à jour du gradient à se produire de manière moins fréquente en attribuant la valeur du batch_size à prendre en compte lors de la regression du modèle.  
Pour cela, on utilise l'argument "batch_size" dans la méthode fit. En précisant un batch_size=1000, cela signifie que :
 - Sur notre total de 56000 échantillons, 56 seront utilisés pour les calculs du gradient
 - Il y aura également 56 itérations à chaque période.
  
    
    
Si nous avions pris le dataset comme entrée, nous aurions eu :
- Un total de 56000 échantillons également
- Chaque période aurait également pris 56 itérations pour se compléter
- Mais 1000 échantillons auraient été utilisés pour le calcul du gradient, au lieu de 56 avec la méthode utilisée.

In [ ]:
# Définition de la fonction de régulation du taux d'apprentissage
def RegulationTauxApprentissage(periode, taux):
  return 1e-8*10**(periode/10)

# Définition de l'optimiseur à utiliser
optimiseur=tf.keras.optimizers.Adam()

# Compile le modèle
model.compile(loss="mse", optimizer=optimiseur, metrics="mse")

# Utilisation de la méthode ModelCheckPoint
CheckPoint = tf.keras.callbacks.ModelCheckpoint("poids.hdf5", monitor='loss', verbose=1, save_best_only=True, save_weights_only = True, mode='auto', save_freq='epoch')

# Entraine le modèle en utilisant notre fonction personnelle de régulation du taux d'apprentissage
historique = model.fit(x=x_train,y=y_train,epochs=100,verbose=1, callbacks=[tf.keras.callbacks.LearningRateScheduler(RegulationTauxApprentissage), CheckPoint],batch_size=batch_size)

In [ ]:
# Construit un vecteur avec les valeurs du taux d'apprentissage à chaque période 
taux = 1e-8*(10**(np.arange(100)/10))

# Affiche l'erreur en fonction du taux d'apprentissage
plt.figure(figsize=(10, 6))
plt.semilogx(taux,historique.history["loss"])
plt.axis([ taux[0], taux[99], 0, 1])
plt.title("Evolution de l'erreur en fonction du taux d'apprentissage")

In [ ]:
# Chargement des poids sauvegardés
model.load_weights("poids.hdf5")

In [ ]:
max_periodes = 10000

# Classe permettant d'arrêter l'entrainement si la variation
# devient plus petite qu'une valeur à choisir sur un nombre
# de périodes à choisir
class StopTrain(keras.callbacks.Callback):
    def __init__(self, delta=0.01,periodes=100, term="loss", logs={}):
      self.n_periodes = 0
      self.periodes = periodes
      self.loss_1 = 100
      self.delta = delta
      self.term = term
    def on_epoch_end(self, epoch, logs={}):
      diff_loss = abs(self.loss_1 - logs[self.term])
      self.loss_1 = logs[self.term]
      if (diff_loss < self.delta):
        self.n_periodes = self.n_periodes + 1
      else:
        self.n_periodes = 0
      if (self.n_periodes == self.periodes):
        print("Arrêt de l'entrainement...")
        self.model.stop_training = True


class PenalizedLoss(tf.keras.losses.Loss):
  def __init__(self,reduction=tf.keras.losses.Reduction.AUTO,name='Penalized_loss'):
    self.reduction = reduction
    self.name = name
    self.y_true_1 = np.linspace(1,1000,1000)
    self.n = 0

  def __call__(self, y_true, y_pred, sample_weight=None):
    if self.n == 0:
      self.n = 1
      error = (tf.keras.metrics.mse(y_true[:,0],y_pred[:,0]))
      self.y_true_1 = y_true[:,0]
      return (error)
    else:
      penalized = tf.keras.metrics.mse(y_true_1,y_pred[:,0])
      error = (tf.keras.metrics.mse(y_true[:,0],y_pred[:,0]))
      return (error + 1/penalized)

def  My_MSE(y_true,y_pred):
  return(tf.keras.metrics.mse(y_true,y_pred)*std.numpy()+mean.numpy())

# Définition des paramètres liés à l'évolution du taux d'apprentissage
lr_schedule = tf.keras.optimizers.schedules.InverseTimeDecay(
    initial_learning_rate=0.001,
    decay_steps=10,
    decay_rate=0.01)

# Définition de l'optimiseur à utiliser
#optimiseur=tf.keras.optimizers.Adam(learning_rate=lr_schedule)

# Utilisation de la méthode ModelCheckPoint
CheckPoint = tf.keras.callbacks.ModelCheckpoint("poids_train.hdf5", monitor='loss', verbose=1, save_best_only=True, save_weights_only = True, mode='auto', save_freq='epoch')

# Compile le modèle
model.compile(loss=PenalizedLoss(), optimizer=optimiseur, metrics=["mse",My_MSE])

# Entraine le modèle, avec une réduction des calculs du gradient
#historique = model.fit(x=x_train,y=y_train,validation_data=(x_val,y_val), epochs=max_periodes,verbose=1, callbacks=[CheckPoint,StopTrain(delta=1e-7,periodes = 10, term="My_MSE")],batch_size=batch_size)

# Entraine le modèle sans réduction de calculs
historique = model.fit(dataset,validation_data=dataset_val, epochs=max_periodes,verbose=1, callbacks=[CheckPoint,StopTrain(delta=1e-8,periodes = 10, term="val_My_MSE")])
#historique = model.fit(dataset, epochs=max_periodes)


In [ ]:
model.load_weights("poids_train.hdf5")

In [ ]:
erreur_entrainement = historique.history["loss"]
erreur_validation = historique.history["val_loss"]

# Affiche l'erreur en fonction de la période
plt.figure(figsize=(10, 6))
plt.plot(np.arange(0,len(erreur_entrainement)),erreur_entrainement, label="Erreurs sur les entrainements")
plt.plot(np.arange(0,len(erreur_entrainement)),erreur_validation, label ="Erreurs sur les validations")
plt.legend()

plt.title("Evolution de l'erreur en fonction de la période")

In [ ]:
# Evaluation du modèle

model.evaluate(dataset)
model.evaluate(dataset_val)


In [ ]:
from google.colab import files
files.download('poids_train.hdf5')

**3. Prédictions**

In [ ]:
# Création des instants d'entrainement et de validation
y_train_timing = serie_entrainement.index[taille_fenetre + horizon - 1:taille_fenetre + horizon - 1+len(y_train)]
y_val_timing = serie_test.index[taille_fenetre + horizon - 1:taille_fenetre + horizon - 1+len(y_val)]

# Calcul des prédictions
pred_ent = model.predict(x_train, verbose=1)
pred_val = model.predict(x_val, verbose=1)

In [ ]:
from statsmodels.tsa.statespace.tools import diff

predictions=[]

for t in range(0,100):
  data_to_predict = serie_entrainement[t:t+taille_fenetre]

  data_to_predict = tf.expand_dims(data_to_predict,axis=1)
  data_to_predict = tf.expand_dims(data_to_predict,axis=0)
  pred = model.predict(data_to_predict)
  predictions.append(pred[0])

In [ ]:
predictions

In [ ]:
import plotly.graph_objects as go

fig = go.Figure()

tmax = len(np.asarray(predictions)[:,0])

# Courbe originale
fig.add_trace(go.Scatter(x=serie_etude.index,y=serie_entrainement[0:taille_fenetre],line=dict(color='blue', width=1),name="diff_reel"))

# Courbes des prédictions d'entrainement
#fig.add_trace(go.Scatter(x=serie_etude.index,y=data_to_predict[:,:,0][0],line=dict(color='green', width=1),name="data_to_predict"))
fig.add_trace(go.Scatter(x=serie_etude.index[taille_fenetre:taille_fenetre+tmax],y=np.asarray(predictions)[:,0],line=dict(color='red', width=1),name="prediction"))

fig.add_trace(go.Scatter(x=serie_etude.index[taille_fenetre:taille_fenetre+tmax],y=serie_entrainement[taille_fenetre:taille_fenetre+tmax],line=dict(color='black', width=1),name="true"))




fig.update_xaxes(rangeslider_visible=True)
yaxis=dict(autorange = True,fixedrange= False)
fig.update_yaxes(yaxis)
fig.show()

**4.Affichage sur une période de 1 heure**

Création d'une série contenant les valeurs originales et les prédictions, synchronisées dans le temps :

In [ ]:
serie_btc_ent_ori = serie_entrainement*std+mean
serie_btc_val_ori = serie_test*std+mean

serie_btc_ent_pred = pd.Series(data=(pred_ent[:,0]*std+mean),index=y_train_timing)
serie_btc_val_pred = pd.Series(data=(pred_val[:,0]*std+mean),index=y_val_timing)

serie_btc_ori = pd.concat([serie_btc_ent_ori,serie_btc_val_ori])
serie_btc_pred = pd.concat([serie_btc_ent_pred,serie_btc_val_pred])

serie_btc_ori = serie_btc_ori.fillna(method="backfill")
serie_btc_pred = serie_btc_pred.fillna(method="backfill")

serie_btc_ent_ori = serie_btc_ent_ori.fillna(method="backfill")
serie_btc_val_ori = serie_btc_val_ori.fillna(method="backfill")
serie_btc_ent_pred = serie_btc_ent_pred.fillna(method="backfill")
serie_btc_val_pred = serie_btc_val_pred.fillna(method="backfill")

frame = {'BTC_ALL' : serie_btc_ori, 'BTC_ENT': serie_btc_ent_ori, 'BTC_VAL' : serie_btc_val_ori, 'BTC_PRED' : serie_btc_pred, 'BTC_PRED_ENT' : serie_btc_ent_pred, 'BTC_PRED_VAL':serie_btc_val_pred}
df_resultats = pd.DataFrame(frame)
df_resultats

In [ ]:
import plotly.graph_objects as go

fig = go.Figure()

# Courbe originale
fig.add_trace(go.Scatter(x=df_resultats.index,y=df_resultats['BTC_ALL'],line=dict(color='blue', width=1),name="Prix BTC"))

# Courbes des prédictions d'entrainement
fig.add_trace(go.Scatter(x=df_resultats.index,y=df_resultats['BTC_PRED_ENT'],line=dict(color='green', width=1),name="Entrainement"))

# Courbe de validation
fig.add_trace(go.Scatter(x=df_resultats.index,y=df_resultats['BTC_PRED_VAL'],line=dict(color='red', width=1),name="Validation"))

fig.update_xaxes(rangeslider_visible=True)
yaxis=dict(autorange = True,fixedrange= False)
fig.update_yaxes(yaxis)
fig.show()

In [ ]:
serie_etude

In [ ]:
from scipy.special import inv_boxcox

In [ ]:
from scipy.integrate import cumtrapz
import numpy as np
import matplotlib.pyplot as plt

def f(x):
    return serie_etude['Open'][0:taille_fenetre+1]

f = np.vectorize(f)

X = np.linspace(0,taille_fenetre+1,taille_fenetre+2)

fv = f(X)
plt.plot(fv)

F = cumtrapz(fv, x=X, initial=0)
plt.plot(F);

In [ ]:
from statsmodels.tsa.statespace.tools import diff

predictions=[]

for t in range(0,200):
  data_to_predict = serie_etude['Open'][t:t+taille_fenetre+1]
  data_to_predict = boxcox(data_to_predict,lam)
  data_to_predict = data_to_predict - trend[t:t+taille_fenetre+1]
  data_to_predict = diff(data_to_predict,1)
  data_to_predict = np.insert(data_to_predict,0,0)

  data_to_predict = tf.expand_dims(data_to_predict,axis=1)
  data_to_predict = tf.expand_dims(data_to_predict,axis=0)
  pred = model.predict(data_to_predict[:,1:,:])
  predictions.append(pred[0])

In [ ]:
np.asarray(predictions)[:,0]

In [ ]:
serie_etude.index[taille_fenetre+1:taille_fenetre+2]

In [ ]:
import plotly.graph_objects as go

fig = go.Figure()

tmax = len(np.asarray(predictions)[:,0])

# Courbe originale
fig.add_trace(go.Scatter(x=serie_etude.index,y=serie_etude['diff'][0:taille_fenetre+1],line=dict(color='blue', width=1),name="diff_reel"))

# Courbes des prédictions d'entrainement
#fig.add_trace(go.Scatter(x=serie_etude.index,y=data_to_predict[:,:,0][0],line=dict(color='green', width=1),name="data_to_predict"))
fig.add_trace(go.Scatter(x=serie_etude.index[taille_fenetre+1:taille_fenetre+1+tmax],y=np.asarray(predictions)[:,0],line=dict(color='red', width=1),name="prediction"))

fig.add_trace(go.Scatter(x=serie_etude.index[taille_fenetre+1:taille_fenetre+1+tmax],y=serie_etude['diff'][taille_fenetre+1:taille_fenetre+1+tmax],line=dict(color='black', width=1),name="true"))




fig.update_xaxes(rangeslider_visible=True)
yaxis=dict(autorange = True,fixedrange= False)
fig.update_yaxes(yaxis)
fig.show()

# Création du modèle LSTM

**1. Création du réseau**

Par défaut, la dimension des vecteurs cachés est de 10 et aucune régularisation n'est utilisée.

In [ ]:
dim_LSTM = 40
l1_reg = 0.0
l2_reg = 0.0

entrees = tf.keras.layers.Input(shape=(taille_fenetre,1))

# Encodeur
s_encodeur = tf.keras.layers.LSTM(dim_LSTM, kernel_regularizer=tf.keras.regularizers.l1_l2(l1=l1_reg,l2=l2_reg))(entrees)
  
# Générateur
sortie = tf.keras.layers.Dense(1,kernel_regularizer=tf.keras.regularizers.l1_l2(l1=l1_reg,l2=l2_reg))(s_encodeur)

# Construction du modèle
model = tf.keras.Model(entrees,sortie)
model.summary()

**2. Optimisation de l'apprentissage**

Pour accélérer le traitement des données, nous n'allons pas utiliser l'intégralité des données pendant la mise à jour du gradient, comme cela a été fait jusqu'à présent (en utilisant le dataset).  
Cette fois-ci, nous allons forcer les mises à jour du gradient à se produire de manière moins fréquente en attribuant la valeur du batch_size à prendre en compte lors de la regression du modèle.  
Pour cela, on utilise l'argument "batch_size" dans la méthode fit. En précisant un batch_size=1000, cela signifie que :
 - Sur notre total de 56000 échantillons, 56 seront utilisés pour les calculs du gradient
 - Il y aura également 56 itérations à chaque période.
  
    
    
Si nous avions pris le dataset comme entrée, nous aurions eu :
- Un total de 56000 échantillons également
- Chaque période aurait également pris 56 itérations pour se compléter
- Mais 1000 échantillons auraient été utilisés pour le calcul du gradient, au lieu de 56 avec la méthode utilisée.

In [ ]:
batch_size = 1000

In [ ]:
# Définition de la fonction de régulation du taux d'apprentissage
def RegulationTauxApprentissage(periode, taux):
  return 1e-8*10**(periode/10)

# Définition de l'optimiseur à utiliser
optimiseur=tf.keras.optimizers.Adam()

# Compile le modèle
model.compile(loss="logcosh", optimizer=optimiseur, metrics="mse")

# Utilisation de la méthode ModelCheckPoint
CheckPoint = tf.keras.callbacks.ModelCheckpoint("poids.hdf5", monitor='loss', verbose=1, save_best_only=True, save_weights_only = True, mode='auto', save_freq='epoch')

# Entraine le modèle en utilisant notre fonction personnelle de régulation du taux d'apprentissage
historique = model.fit(x=x_train,y=y_train,epochs=100,verbose=1, callbacks=[tf.keras.callbacks.LearningRateScheduler(RegulationTauxApprentissage), CheckPoint],batch_size=batch_size)

In [ ]:
# Construit un vecteur avec les valeurs du taux d'apprentissage à chaque période 
taux = 1e-8*(10**(np.arange(100)/10))

# Affiche l'erreur en fonction du taux d'apprentissage
plt.figure(figsize=(10, 6))
plt.semilogx(taux,historique.history["loss"])
plt.axis([ taux[30], taux[99], 0, 0.04])
plt.title("Evolution de l'erreur en fonction du taux d'apprentissage")

In [ ]:
# Chargement des poids sauvegardés
model.load_weights("poids.hdf5")

In [ ]:
max_periodes = 1000

# Classe permettant d'arrêter l'entrainement si la variation
# devient plus petite qu'une valeur à choisir sur un nombre
# de périodes à choisir
class StopTrain(keras.callbacks.Callback):
    def __init__(self, delta=0.01,periodes=100, term="loss", logs={}):
      self.n_periodes = 0
      self.periodes = periodes
      self.loss_1 = 100
      self.delta = delta
      self.term = term
    def on_epoch_end(self, epoch, logs={}):
      diff_loss = abs(self.loss_1 - logs[self.term])
      self.loss_1 = logs[self.term]
      if (diff_loss < self.delta):
        self.n_periodes = self.n_periodes + 1
      else:
        self.n_periodes = 0
      if (self.n_periodes == self.periodes):
        print("Arrêt de l'entrainement...")
        self.model.stop_training = True

def  My_MSE(y_true,y_pred):
  return(tf.keras.metrics.mse(y_true,y_pred)*std.numpy()+mean.numpy())
  
# Définition des paramètres liés à l'évolution du taux d'apprentissage
lr_schedule = tf.keras.optimizers.schedules.InverseTimeDecay(
    initial_learning_rate=0.02,
    decay_steps=10,
    decay_rate=0.01)

# Définition de l'optimiseur à utiliser
optimiseur=tf.keras.optimizers.Adam(learning_rate=lr_schedule)


# Utilisation de la méthode ModelCheckPoint
CheckPoint = tf.keras.callbacks.ModelCheckpoint("poids_train.hdf5", monitor='loss', verbose=1, save_best_only=True, save_weights_only = True, mode='auto', save_freq='epoch')

# Compile le modèle
model.compile(loss="logcosh", optimizer=optimiseur, metrics=["mse",My_MSE])

# Entraine le modèle, avec une réduction des calculs du gradient
historique = model.fit(x=x_train,y=y_train,validation_data=(x_val,y_val), epochs=max_periodes,verbose=1, callbacks=[CheckPoint,StopTrain(delta=0.005,periodes = 10, term="My_MSE")],batch_size=batch_size)

# Entraine le modèle sans réduction de calculs
#historique = model.fit(dataset,validation_data=dataset_val, epochs=max_periodes,verbose=1, callbacks=[CheckPoint,StopTrain(delta=0.1,periodes = 10, term="val_My_MSE")])


In [ ]:
erreur_entrainement = historique.history["loss"]
erreur_validation = historique.history["val_loss"]

# Affiche l'erreur en fonction de la période
plt.figure(figsize=(10, 6))
plt.plot(np.arange(0,len(erreur_entrainement)),erreur_entrainement, label="Erreurs sur les entrainements")
plt.plot(np.arange(0,len(erreur_entrainement)),erreur_validation, label ="Erreurs sur les validations")
plt.legend()

plt.title("Evolution de l'erreur en fonction de la période")

In [ ]:
# Evaluation du modèle
# Avec le modèle type encodeur/décodeur :
# 56/56 [==============================] - 7s 113ms/step - loss: 1.1795e-04 - mse: 2.3664e-04 - My_MSE: 2712.2415
# 14/14 [==============================] - 2s 113ms/step - loss: 0.3602 - mse: 1.9360 - My_MSE: 9650.2637
model.evaluate(dataset)
model.evaluate(dataset_val)


**3. Prédictions**

In [ ]:
# Création des instants d'entrainement et de validation
y_train_timing = serie_entrainement.index[taille_fenetre + horizon - 1:taille_fenetre + horizon - 1+len(y_train)]
y_val_timing = serie_test.index[taille_fenetre + horizon - 1:taille_fenetre + horizon - 1+len(y_val)]

# Calcul des prédictions
pred_ent = model.predict(x_train, verbose=1)
pred_val = model.predict(x_val, verbose=1)

**4.Affichage sur une période de 1 heure**

Création d'une série contenant les valeurs originales et les prédictions, synchronisées dans le temps :

In [ ]:
serie_btc_ent_ori = serie_entrainement*std+mean
serie_btc_val_ori = serie_test*std+mean

serie_btc_ent_pred = pd.Series(data=(pred_ent[:,0]*std+mean),index=y_train_timing)
serie_btc_val_pred = pd.Series(data=(pred_val[:,0]*std+mean),index=y_val_timing)

serie_btc_ori = pd.concat([serie_btc_ent_ori,serie_btc_val_ori])
serie_btc_pred = pd.concat([serie_btc_ent_pred,serie_btc_val_pred])

serie_btc_ori = serie_btc_ori.fillna(method="backfill")
serie_btc_pred = serie_btc_pred.fillna(method="backfill")

serie_btc_ent_ori = serie_btc_ent_ori.fillna(method="backfill")
serie_btc_val_ori = serie_btc_val_ori.fillna(method="backfill")
serie_btc_ent_pred = serie_btc_ent_pred.fillna(method="backfill")
serie_btc_val_pred = serie_btc_val_pred.fillna(method="backfill")

frame = {'BTC_ALL' : serie_btc_ori, 'BTC_ENT': serie_btc_ent_ori, 'BTC_VAL' : serie_btc_val_ori, 'BTC_PRED' : serie_btc_pred, 'BTC_PRED_ENT' : serie_btc_ent_pred, 'BTC_PRED_VAL':serie_btc_val_pred}
df_resultats = pd.DataFrame(frame)
df_resultats

In [ ]:
date_separation

In [ ]:
df_resultats.loc[date_separation-pd.Timedelta(hours=7):date_separation+pd.Timedelta(hours=7)]

In [ ]:
import plotly.graph_objects as go

fig = go.Figure()

# Courbe originale
fig.add_trace(go.Scatter(x=df_resultats.index,y=df_resultats['BTC_ALL'],line=dict(color='blue', width=1),name="Prix BTC"))

# Courbes des prédictions d'entrainement
fig.add_trace(go.Scatter(x=df_resultats.index,y=df_resultats['BTC_PRED_ENT'],line=dict(color='green', width=1),name="Entrainement"))

# Courbe de validation
fig.add_trace(go.Scatter(x=df_resultats.index,y=df_resultats['BTC_PRED_VAL'],line=dict(color='red', width=1),name="Validation"))

fig.update_xaxes(rangeslider_visible=True)
yaxis=dict(autorange = True,fixedrange= False)
fig.update_yaxes(yaxis)
fig.show()

**5. Détection de l'augmentation des erreurs dans la zone de validation**

On peut imaginer devoir suivre en temps réel l'évolution des prédictions afin de détecter lorsque le modèle n'est plus valide.

In [ ]:
# Détection de la date où commencent les anomalies
# sur la période de validation

# Construit les dataframe pour calculer les erreurs
df_error_btc_ent = df_resultats[['BTC_ENT','BTC_PRED_ENT']].dropna()
df_error_btc_val = df_resultats[['BTC_VAL','BTC_PRED_VAL']].dropna()

# Calcul des erreurs relatives sur les entrainements et les validations
erreur_ent = abs(np.asarray(df_error_btc_ent['BTC_ENT']) - np.asarray(df_error_btc_ent['BTC_PRED_ENT']))/np.asarray(df_error_btc_ent['BTC_ENT'])*100.0
erreur_val = abs(np.asarray(df_error_btc_val['BTC_VAL']) - np.asarray(df_error_btc_val['BTC_PRED_VAL']))/np.asarray(df_error_btc_val['BTC_VAL'])*100.0

# Erreur relative moyenne sur les entrainements
erreur_ent_mape = tf.keras.metrics.mape(np.asarray(df_error_btc_ent['BTC_ENT']),np.asarray(df_error_btc_ent['BTC_PRED_ENT']))
erreur_ent_mape_max = np.amax(erreur_ent)

# Calcul le nombre d'anomalies sur les entrainements avec le ratio spécifié
nbr_anomalies_ent = np.count_nonzero(erreur_ent>erreur_ent_mape)

# Calcul du ratio d'anomalies sur la période d'entrainement
ratio_ent = np.count_nonzero(erreur_ent>erreur_ent_mape)/erreur_ent.size

# Recherche de la date à partir de laquelle on dépasse l
n_erreurs = 0
for i in range(5,erreur_val.size):
  erreur_val_ = abs(np.asarray(df_error_btc_val['BTC_VAL'][i]) - np.asarray(df_error_btc_val['BTC_PRED_VAL'][i]))/np.asarray(df_error_btc_val['BTC_VAL'][i])*100.0
  seuil = erreur_ent_mape_max*1
  if erreur_val_ > seuil:
    n_erreurs = n_erreurs + 1
    if n_erreurs == 5:
      index = df_error_btc_val.index[i]
      break
  else:
    n_erreurs = 0
print(index)

In [ ]:
import plotly.graph_objects as go

fig = go.Figure()

# Courbe originale
fig.add_trace(go.Scatter(x=df_resultats.index,y=df_resultats['BTC_ALL'],line=dict(color='blue', width=1),name="Prix BTC"))

# Courbes des prédictions d'entrainement
fig.add_trace(go.Scatter(x=df_resultats.index,y=df_resultats['BTC_PRED_ENT'],line=dict(color='green', width=1),name="Entrainement"))

# Courbe de validation
fig.add_trace(go.Scatter(x=df_resultats.index,y=df_resultats['BTC_PRED_VAL'],line=dict(color='red', width=1),name="Validation"))

# Délimite la zone d'erreur
fig.add_shape(type="line",x0=index,y0=0,x1=index,y1=np.amax(df_resultats['BTC_ALL']),name="Zone d'erreur")

# Affiche les courbes
fig.update_xaxes(rangeslider_visible=True)
yaxis=dict(autorange = True,fixedrange= False)
fig.update_yaxes(yaxis)
fig.show()

**5.Affichage sur une période de 1 jour**

In [ ]:
# Echantillonage des données
time = "1D"

df_resultats = df_resultats.resample(time).asfreq()
df_resultats

In [ ]:
import plotly.graph_objects as go

fig = go.Figure()

# Courbe originale
fig.add_trace(go.Scatter(x=df_resultats.index,y=df_resultats['BTC_ALL'],line=dict(color='blue', width=1),name="Prix BTC"))

# Courbes des prédictions d'entrainement
fig.add_trace(go.Scatter(x=df_resultats.index,y=df_resultats['BTC_PRED_ENT'],line=dict(color='green', width=1),name="Entrainement"))

# Courbe de validation
fig.add_trace(go.Scatter(x=df_resultats.index,y=df_resultats['BTC_PRED_VAL'],line=dict(color='red', width=1),name="Validation"))

# Délimite la zone d'erreur
fig.add_shape(type="line",x0=index,y0=0,x1=index,y1=np.amax(df_resultats['BTC_ALL']),name="Zone d'erreur")

# Affiche les courbes
fig.update_xaxes(rangeslider_visible=True)
yaxis=dict(autorange = True,fixedrange= False)
fig.update_yaxes(yaxis)
fig.show()

**6.Synthèse des erreurs sur les différentes zones**

In [ ]:
# Echantillonage des données
time = "1H"

df_resultats = df_resultats.resample(time).asfreq()
df_resultats

In [ ]:
# Détection de la date où commencnte les anomalies
# sur la période de validation

# Construit les dataframe pour calculer les erreurs
df_error_btc_ent = df_resultats[['BTC_ENT','BTC_PRED_ENT']].dropna()
df_error_btc_val = df_resultats[['BTC_VAL','BTC_PRED_VAL']].dropna()


In [ ]:
# Erreurs d'entrainement
mae_ent = tf.keras.metrics.mean_absolute_error(np.asarray(df_error_btc_ent['BTC_ENT']),np.asarray(df_error_btc_ent['BTC_PRED_ENT'])).numpy()
mse_ent = tf.keras.metrics.mean_squared_error(np.asarray(df_error_btc_ent['BTC_ENT']),np.asarray(df_error_btc_ent['BTC_PRED_ENT'])).numpy()
mape_ent = tf.keras.metrics.mape(np.asarray(df_error_btc_ent['BTC_ENT']),np.asarray(df_error_btc_ent['BTC_PRED_ENT'])).numpy()

# Erreurs de validation
mae_val = tf.keras.metrics.mean_absolute_error(np.asarray(df_error_btc_val['BTC_VAL']),np.asarray(df_error_btc_val['BTC_PRED_VAL'])).numpy()
mse_val = tf.keras.metrics.mean_squared_error(np.asarray(df_error_btc_val['BTC_VAL']),np.asarray(df_error_btc_val['BTC_PRED_VAL'])).numpy()
mape_val = tf.keras.metrics.mape(np.asarray(df_error_btc_val['BTC_VAL']),np.asarray(df_error_btc_val['BTC_PRED_VAL'])).numpy()

# Erreurs sur la zone valide de validation
mae_val_ok = tf.keras.metrics.mean_absolute_error(df_error_btc_val['BTC_VAL'][:index],np.asarray(df_error_btc_val['BTC_PRED_VAL'][:index])).numpy()
mse_val_ok = tf.keras.metrics.mean_squared_error(df_error_btc_val['BTC_VAL'][:index],np.asarray(df_error_btc_val['BTC_PRED_VAL'][:index])).numpy()
mape_val_ok = tf.keras.metrics.mape(df_error_btc_val['BTC_VAL'][:index],np.asarray(df_error_btc_val['BTC_PRED_VAL'][:index])).numpy()


print("Erreur mae entrainement %s" %mae_ent)
print("Erreur mse entrainement : %s" %mse_ent)
print("Erreur mape entrainement : %s\n" %mape_ent)

print("Erreur mae validation %s" %mae_val)
print("Erreur mse validation : %s" %mse_val)
print("Erreur mape entrainement : %s\n" %mape_val)

print("Erreur mae validation zone OK %s" %mae_val_ok)
print("Erreur mse validation zone OK: %s" %mse_val_ok)
print("Erreur mape validation zone OK: %s" %mape_val_ok)

# Création du modèle LSTM avec auto-attention

**1. Création du réseau**

Par défaut, la dimension des vecteurs cachés est de 10 et aucune régularisation n'est utilisée.

In [ ]:
# Classe d'auto-attention
# Applique les poids de la matrice d'attention sur les vecteurs de la couche récurrente

# Importe le Backend de Keras
from keras import backend as K

# Définit une nouvelle classe Couche_Attention
# Héritée de la classe Layer de Keras

class Couche_Auto_Attention(tf.keras.layers.Layer):
  # Fonction d'initialisation de la classe d'attention
  def __init__(self,dim_att,nbr_hop):
    self.dim_att = dim_att          # Dimension du vecteur d'attention
    self.nbr_hop = nbr_hop
    super().__init__()              # Appel du __init__() de la classe Layer
  
  def build(self,input_shape):
    self.W = self.add_weight(shape=(self.dim_att,input_shape[2]),initializer='glorot_uniform',name="W")
    self.U = self.add_weight(shape=(self.nbr_hop,self.dim_att),initializer='glorot_uniform',name="U")
    super().build(input_shape)        # Appel de la méthode build()

  # Définit la logique de la couche d'attention
  # Arguments :   x : Tenseur d'entrée de dimension (None, nbr_v,dim)
  def call(self,x):
    # Calcul de la matrice XH contenant les
    # représentations cachées des vecteurs
    # issus de la couche GRU
    xt = tf.transpose(x,perm=[0,2,1])           # (None,20,40) => (None,40,20)
    Xh = tf.matmul(self.W,xt)                   # (#Att,40)x(None,40,20) = (None,#Att,20)
    Xh = K.tanh(Xh)                             # Xh = (None,#Att,20)

    # Calcul de la matrice des poids d'attention normalisés
    A = tf.matmul(self.U,Xh)                    # (#hop,#Att)x(None,#Att,20) = (None,#Att,20)
    A = tf.keras.activations.softmax(A,axis=2)  # (None,#Att,20)

    # Calcul de la matrice des vecteur d'attentions
    sortie = tf.matmul(A,x)                     # (None,#Att,20)x(None,20,40) = (None,#Att,40)
    return tf.keras.layers.Flatten()(sortie)

In [ ]:
dim_LSTM = 100
l1_reg = 0.0
l2_reg = 0.00
nbr_hop = 20

# Définition de l'entrée du modèle
entrees = tf.keras.layers.Input(shape=(taille_fenetre,1))

# Encodeur
s_encodeur = tf.keras.layers.LSTM(dim_LSTM,return_sequences=True,recurrent_regularizer=tf.keras.regularizers.l2(1e-5))(entrees)
s_attention = Couche_Auto_Attention(dim_att=dim_LSTM,nbr_hop=nbr_hop)(s_encodeur)

# Décodeur
s_decodeur = tf.keras.layers.Dense(dim_LSTM*nbr_hop,activation="tanh")(s_attention)
s_decodeur = tf.keras.layers.Concatenate()([s_decodeur,s_attention])

# Générateur
sortie = tf.keras.layers.Dense(1)(s_decodeur)

# Construction du modèle
model = tf.keras.Model(entrees,sortie)

model.summary()

**2. Optimisation de l'apprentissage**

Pour accélérer le traitement des données, nous n'allons pas utiliser l'intégralité des données pendant la mise à jour du gradient, comme cela a été fait jusqu'à présent (en utilisant le dataset).  
Cette fois-ci, nous allons forcer les mises à jour du gradient à se produire de manière moins fréquente en attribuant la valeur du batch_size à prendre en compte lors de la regression du modèle.  
Pour cela, on utilise l'argument "batch_size" dans la méthode fit. En précisant un batch_size=1000, cela signifie que :
 - Sur notre total de 56000 échantillons, 56 seront utilisés pour les calculs du gradient
 - Il y aura également 56 itérations à chaque période.
  
    
    
Si nous avions pris le dataset comme entrée, nous aurions eu :
- Un total de 56000 échantillons également
- Chaque période aurait également pris 56 itérations pour se compléter
- Mais 1000 échantillons auraient été utilisés pour le calcul du gradient, au lieu de 56 avec la méthode utilisée.

In [ ]:
batch_size = 1000

In [ ]:
# Définition de la fonction de régulation du taux d'apprentissage
def RegulationTauxApprentissage(periode, taux):
  return 1e-8*10**(periode/10)

# Définition de l'optimiseur à utiliser
optimiseur=tf.keras.optimizers.Adam()

# Compile le modèle
model.compile(loss="mse", optimizer=optimiseur, metrics="mse")

# Utilisation de la méthode ModelCheckPoint
CheckPoint = tf.keras.callbacks.ModelCheckpoint("poids.hdf5", monitor='loss', verbose=1, save_best_only=True, save_weights_only = True, mode='auto', save_freq='epoch')

# Entraine le modèle en utilisant notre fonction personnelle de régulation du taux d'apprentissage
historique = model.fit(x=x_train,y=y_train,epochs=100,verbose=1, callbacks=[tf.keras.callbacks.LearningRateScheduler(RegulationTauxApprentissage), CheckPoint],batch_size=batch_size)

In [ ]:
# Construit un vecteur avec les valeurs du taux d'apprentissage à chaque période 
taux = 1e-8*(10**(np.arange(100)/10))

# Affiche l'erreur en fonction du taux d'apprentissage
plt.figure(figsize=(10, 6))
plt.semilogx(taux,historique.history["loss"])
plt.axis([ taux[0], taux[99], 0, 2])
plt.title("Evolution de l'erreur en fonction du taux d'apprentissage")

In [ ]:
# Chargement des poids sauvegardés
model.load_weights("poids.hdf5")

In [ ]:
max_periodes = 1000

# Classe permettant d'arrêter l'entrainement si la variation
# devient plus petite qu'une valeur à choisir sur un nombre
# de périodes à choisir
class StopTrain(keras.callbacks.Callback):
    def __init__(self, delta=0.01,periodes=100, term="loss", logs={}):
      self.n_periodes = 0
      self.periodes = periodes
      self.loss_1 = 100
      self.delta = delta
      self.term = term
    def on_epoch_end(self, epoch, logs={}):
      diff_loss = abs(self.loss_1 - logs[self.term])
      self.loss_1 = logs[self.term]
      if (diff_loss < self.delta):
        self.n_periodes = self.n_periodes + 1
      else:
        self.n_periodes = 0
      if (self.n_periodes == self.periodes):
        print("Arrêt de l'entrainement...")
        self.model.stop_training = True

def  My_MSE(y_true,y_pred):
  return(tf.keras.metrics.mse(y_true,y_pred)*std.numpy()+mean.numpy())
  
# Définition des paramètres liés à l'évolution du taux d'apprentissage
lr_schedule = tf.keras.optimizers.schedules.InverseTimeDecay(
    initial_learning_rate=0.001,
    decay_steps=10,
    decay_rate=0.1)

# Définition de l'optimiseur à utiliser
optimiseur=tf.keras.optimizers.Adam(learning_rate=lr_schedule)


# Utilisation de la méthode ModelCheckPoint
CheckPoint = tf.keras.callbacks.ModelCheckpoint("poids_train.hdf5", monitor='loss', verbose=1, save_best_only=True, save_weights_only = True, mode='auto', save_freq='epoch')

# Compile le modèle
model.compile(loss="mse", optimizer=optimiseur, metrics=["mse",My_MSE])

# Entraine le modèle, avec une réduction des calculs du gradient
historique = model.fit(x=x_train,y=y_train,validation_data=(x_val,y_val), epochs=max_periodes,verbose=1, callbacks=[CheckPoint,StopTrain(delta=0.0005,periodes = 10, term="My_MSE")],batch_size=batch_size)

# Entraine le modèle sans réduction de calculs
#historique = model.fit(dataset,validation_data=dataset_val, epochs=max_periodes,verbose=1, callbacks=[CheckPoint,StopTrain(delta=0.1,periodes = 10, term="val_My_MSE")])


In [ ]:
model.load_weights('poids_train.hdf5')

In [ ]:
erreur_entrainement = historique.history["loss"]
erreur_validation = historique.history["val_loss"]

# Affiche l'erreur en fonction de la période
plt.figure(figsize=(10, 6))
plt.plot(np.arange(0,len(erreur_entrainement)),erreur_entrainement, label="Erreurs sur les entrainements")
plt.plot(np.arange(0,len(erreur_entrainement)),erreur_validation, label ="Erreurs sur les validations")
plt.legend()

plt.title("Evolution de l'erreur en fonction de la période")

In [ ]:
# Evaluation du modèle

model.evaluate(dataset)
model.evaluate(dataset_val)


**3. Prédictions**

In [ ]:
# Création des instants d'entrainement et de validation
y_train_timing = serie_entrainement_x.index[taille_fenetre + horizon - 1:taille_fenetre + horizon - 1+len(y_train)]
y_val_timing = serie_test_x.index[taille_fenetre + horizon - 1:taille_fenetre + horizon - 1+len(y_val)]

# Calcul des prédictions
pred_ent = model.predict(x_train, verbose=1)
pred_val = model.predict(x_val, verbose=1)

**4.Affichage sur une période de 1 heure**

Création d'une série contenant les valeurs originales et les prédictions, synchronisées dans le temps :

In [ ]:
serie_btc_ent_ori = serie_entrainement_y*std+mean
serie_btc_val_ori = serie_test_y*std+mean

serie_btc_ent_pred = pd.Series(data=(pred_ent[:,0]*std+mean),index=y_train_timing)
serie_btc_val_pred = pd.Series(data=(pred_val[:,0]*std+mean),index=y_val_timing)

serie_btc_ori = pd.concat([serie_btc_ent_ori,serie_btc_val_ori])
serie_btc_pred = pd.concat([serie_btc_ent_pred,serie_btc_val_pred])

serie_btc_ori = serie_btc_ori.fillna(method="backfill")
serie_btc_pred = serie_btc_pred.fillna(method="backfill")

serie_btc_ent_ori = serie_btc_ent_ori.fillna(method="backfill")
serie_btc_val_ori = serie_btc_val_ori.fillna(method="backfill")
serie_btc_ent_pred = serie_btc_ent_pred.fillna(method="backfill")
serie_btc_val_pred = serie_btc_val_pred.fillna(method="backfill")

frame = {'BTC_ALL' : serie_btc_ori, 'BTC_ENT': serie_btc_ent_ori, 'BTC_VAL' : serie_btc_val_ori, 'BTC_PRED' : serie_btc_pred, 'BTC_PRED_ENT' : serie_btc_ent_pred, 'BTC_PRED_VAL':serie_btc_val_pred}
df_resultats = pd.DataFrame(frame)
df_resultats

In [ ]:
date_separation

In [ ]:
df_resultats.loc[date_separation-pd.Timedelta(hours=7):date_separation+pd.Timedelta(hours=7)]

In [ ]:
import plotly.graph_objects as go

fig = go.Figure()

# Courbe originale
fig.add_trace(go.Scatter(x=df_resultats.index,y=df_resultats['BTC_ALL'],line=dict(color='blue', width=1),name="Prix BTC"))

# Courbes des prédictions d'entrainement
fig.add_trace(go.Scatter(x=df_resultats.index,y=df_resultats['BTC_PRED_ENT'],line=dict(color='green', width=1),name="Entrainement"))

# Courbe de validation
fig.add_trace(go.Scatter(x=df_resultats.index,y=df_resultats['BTC_PRED_VAL'],line=dict(color='red', width=1),name="Validation"))

fig.update_xaxes(rangeslider_visible=True)
yaxis=dict(autorange = True,fixedrange= False)
fig.update_yaxes(yaxis)
fig.show()

**5. Détection de l'augmentation des erreurs dans la zone de validation**

On peut imaginer devoir suivre en temps réel l'évolution des prédictions afin de détecter lorsque le modèle n'est plus valide.

In [ ]:
# Détection de la date où commencent les anomalies
# sur la période de validation

# Construit les dataframe pour calculer les erreurs
df_error_btc_ent = df_resultats[['BTC_ENT','BTC_PRED_ENT']].dropna()
df_error_btc_val = df_resultats[['BTC_VAL','BTC_PRED_VAL']].dropna()

# Calcul des erreurs relatives sur les entrainements et les validations
erreur_ent = abs(np.asarray(df_error_btc_ent['BTC_ENT']) - np.asarray(df_error_btc_ent['BTC_PRED_ENT']))/np.asarray(df_error_btc_ent['BTC_ENT'])*100.0
erreur_val = abs(np.asarray(df_error_btc_val['BTC_VAL']) - np.asarray(df_error_btc_val['BTC_PRED_VAL']))/np.asarray(df_error_btc_val['BTC_VAL'])*100.0

# Erreur relative moyenne sur les entrainements
erreur_ent_mape = tf.keras.metrics.mape(np.asarray(df_error_btc_ent['BTC_ENT']),np.asarray(df_error_btc_ent['BTC_PRED_ENT']))
erreur_ent_mape_max = np.amax(erreur_ent)

# Calcul le nombre d'anomalies sur les entrainements avec le ratio spécifié
nbr_anomalies_ent = np.count_nonzero(erreur_ent>erreur_ent_mape)

# Calcul du ratio d'anomalies sur la période d'entrainement
ratio_ent = np.count_nonzero(erreur_ent>erreur_ent_mape)/erreur_ent.size

# Recherche de la date à partir de laquelle on dépasse l
n_erreurs = 0
for i in range(5,erreur_val.size):
  erreur_val_ = abs(np.asarray(df_error_btc_val['BTC_VAL'][i]) - np.asarray(df_error_btc_val['BTC_PRED_VAL'][i]))/np.asarray(df_error_btc_val['BTC_VAL'][i])*100.0
  seuil = erreur_ent_mape_max*1
  if erreur_val_ > seuil:
    n_erreurs = n_erreurs + 1
    if n_erreurs == 1:
      index = df_error_btc_val.index[i]
      break
  else:
    n_erreurs = 0
print(index)


In [ ]:
import plotly.graph_objects as go

fig = go.Figure()

# Courbe originale
fig.add_trace(go.Scatter(x=df_resultats.index,y=df_resultats['BTC_ALL'],line=dict(color='blue', width=1),name="Prix BTC"))

# Courbes des prédictions d'entrainement
fig.add_trace(go.Scatter(x=df_resultats.index,y=df_resultats['BTC_PRED_ENT'],line=dict(color='green', width=1),name="Entrainement"))

# Courbe de validation
fig.add_trace(go.Scatter(x=df_resultats.index,y=df_resultats['BTC_PRED_VAL'],line=dict(color='red', width=1),name="Validation"))

# Délimite la zone d'erreur
fig.add_shape(type="line",x0=index,y0=0,x1=index,y1=np.amax(df_resultats['BTC_ALL']),name="Zone d'erreur")

# Affiche les courbes
fig.update_xaxes(rangeslider_visible=True)
yaxis=dict(autorange = True,fixedrange= False)
fig.update_yaxes(yaxis)
fig.show()

**5.Affichage sur une période de 1 jour**

In [ ]:
# Echantillonage des données
time = "1D"

df_resultats = df_resultats.resample(time).asfreq()
df_resultats

In [ ]:
import plotly.graph_objects as go

fig = go.Figure()

# Courbe originale
fig.add_trace(go.Scatter(x=df_resultats.index,y=df_resultats['BTC_ALL'],line=dict(color='blue', width=1),name="Prix BTC"))

# Courbes des prédictions d'entrainement
fig.add_trace(go.Scatter(x=df_resultats.index,y=df_resultats['BTC_PRED_ENT'],line=dict(color='green', width=1),name="Entrainement"))

# Courbe de validation
fig.add_trace(go.Scatter(x=df_resultats.index,y=df_resultats['BTC_PRED_VAL'],line=dict(color='red', width=1),name="Validation"))

# Délimite la zone d'erreur
fig.add_shape(type="line",x0=index,y0=0,x1=index,y1=np.amax(df_resultats['BTC_ALL']),name="Zone d'erreur")

# Affiche les courbes
fig.update_xaxes(rangeslider_visible=True)
yaxis=dict(autorange = True,fixedrange= False)
fig.update_yaxes(yaxis)
fig.show()

**6.Synthèse des erreurs sur les différentes zones**

In [ ]:
# Echantillonage des données
time = "1H"

df_resultats = df_resultats.resample(time).asfreq()
df_resultats

In [ ]:
# Détection de la date où commencnte les anomalies
# sur la période de validation

# Construit les dataframe pour calculer les erreurs
df_error_btc_ent = df_resultats[['BTC_ENT','BTC_PRED_ENT']].dropna()
df_error_btc_val = df_resultats[['BTC_VAL','BTC_PRED_VAL']].dropna()


In [ ]:
# Erreurs d'entrainement
mae_ent = tf.keras.metrics.mean_absolute_error(np.asarray(df_error_btc_ent['BTC_ENT']),np.asarray(df_error_btc_ent['BTC_PRED_ENT'])).numpy()
mse_ent = tf.keras.metrics.mean_squared_error(np.asarray(df_error_btc_ent['BTC_ENT']),np.asarray(df_error_btc_ent['BTC_PRED_ENT'])).numpy()
mape_ent = tf.keras.metrics.mape(np.asarray(df_error_btc_ent['BTC_ENT']),np.asarray(df_error_btc_ent['BTC_PRED_ENT'])).numpy()

# Erreurs de validation
mae_val = tf.keras.metrics.mean_absolute_error(np.asarray(df_error_btc_val['BTC_VAL']),np.asarray(df_error_btc_val['BTC_PRED_VAL'])).numpy()
mse_val = tf.keras.metrics.mean_squared_error(np.asarray(df_error_btc_val['BTC_VAL']),np.asarray(df_error_btc_val['BTC_PRED_VAL'])).numpy()
mape_val = tf.keras.metrics.mape(np.asarray(df_error_btc_val['BTC_VAL']),np.asarray(df_error_btc_val['BTC_PRED_VAL'])).numpy()

# Erreurs sur la zone valide de validation
mae_val_ok = tf.keras.metrics.mean_absolute_error(df_error_btc_val['BTC_VAL'][:index],np.asarray(df_error_btc_val['BTC_PRED_VAL'][:index])).numpy()
mse_val_ok = tf.keras.metrics.mean_squared_error(df_error_btc_val['BTC_VAL'][:index],np.asarray(df_error_btc_val['BTC_PRED_VAL'][:index])).numpy()
mape_val_ok = tf.keras.metrics.mape(df_error_btc_val['BTC_VAL'][:index],np.asarray(df_error_btc_val['BTC_PRED_VAL'][:index])).numpy()


print("Erreur mae entrainement %s" %mae_ent)
print("Erreur mse entrainement : %s" %mse_ent)
print("Erreur mape entrainement : %s\n" %mape_ent)

print("Erreur mae validation %s" %mae_val)
print("Erreur mse validation : %s" %mse_val)
print("Erreur mape entrainement : %s\n" %mape_val)

print("Erreur mae validation zone OK %s" %mae_val_ok)
print("Erreur mse validation zone OK: %s" %mse_val_ok)
print("Erreur mape validation zone OK: %s" %mape_val_ok)

# Création du modèle End-To-End Network

In [ ]:
def prepare_dataset_XY(serie, taille_fenetre, batch_size, nbr_sequences):
  dataset = tf.data.Dataset.from_tensor_slices(serie)
  dataset = dataset.window(taille_fenetre+1, shift=1, drop_remainder=True)
  dataset = dataset.flat_map(lambda x: x.batch(taille_fenetre + 1))
  dataset = dataset.window(nbr_sequences+1, shift=1, drop_remainder=True)
  dataset = dataset.flat_map(lambda x: x.batch(nbr_sequences+1,drop_remainder=True))
  dataset = dataset.map(lambda x: [(tf.slice(x,[0,0],[nbr_sequences,taille_fenetre]),                           # (30;20)       [((30,20),(20)),(1)]
                                   tf.squeeze(tf.slice(x,[nbr_sequences,0],[1,taille_fenetre]),axis=0)),        # (20)
                                   tf.squeeze(tf.slice(x,[nbr_sequences,taille_fenetre],[1,1]),axis=0)])        # (1)
  dataset = dataset.batch(batch_size,drop_remainder=True)
  return dataset

In [ ]:
# Définition des caractéristiques du dataset que l'on souhaite créer
taille_fenetre = 15
batch_size = 1000
Nbr_Sequences = 10

dataset = prepare_dataset_XY(serie_entrainement,taille_fenetre,batch_size,Nbr_Sequences)              # 56x((1000,10,15),(1000,15)),(1000,1)
dataset_val = prepare_dataset_XY(serie_test,taille_fenetre,batch_size,Nbr_Sequences)              # 56x((1000,10,15),(1000,15)),(1000,1)


In [ ]:
len(list(dataset.as_numpy_iterator()))

In [ ]:
for element in dataset.take(1):
  print(element)

In [ ]:
# Extrait les X,Y du dataset
#56x((1000,4,1),(1000,1)) => (56*1000,4,1) ; (56*1000,1)

x,y = tuple(zip(*dataset))

# Recombine les données
# (56,1000,4,1) => (56*128,4,1)
# (56,1000,1) => (56*128,1)
x_train = np.asarray(tf.reshape(np.asarray(x,dtype=np.float32),shape=(np.asarray(x).shape[0]*np.asarray(x).shape[1],taille_fenetre,1)))
y_train = np.asarray(tf.reshape(np.asarray(y,dtype=np.float32),shape=(np.asarray(y).shape[0]*np.asarray(y).shape[1])))

# Affiche les formats
print(x_train.shape)
print(y_train.shape)

**1. Création du réseau**

Par défaut, la dimension des vecteurs cachés est de 10 et aucune régularisation n'est utilisée.

In [ ]:
# Définition du de la couche du modèle
# End-to-End Memory Network
# Epaquetage des données avec le dernier état caché d'une couche GRU

from keras import backend as K

class Couche_End_to_End_MN(tf.keras.layers.Layer):
  # Fonction d'initialisation de la classe d'attention
  # dim_GRU : Dimension des vecteurs GRU
  # x : Séquences à mémoriser (batch_size, Nbr_Sequence, taille_fenetre)
  # Fonction de la couche lambda d'entrée
  def __init__(self,dim_GRU,regul=0.0):
    self.dim_GRU = dim_GRU
    self.regul = regul
    super().__init__()          # Appel du __init__() de la classe Layer
  
  def build(self,input_shape):
    # Définition des couches GRU pour traiter les séquences d'entrée
    self.couche_GRU_A = tf.keras.layers.GRU(self.dim_GRU,kernel_regularizer=tf.keras.regularizers.l2(self.regul))
    self.couche_GRU_B = tf.keras.layers.GRU(self.dim_GRU,kernel_regularizer=tf.keras.regularizers.l2(self.regul))
    self.couche_GRU_C = tf.keras.layers.GRU(self.dim_GRU,kernel_regularizer=tf.keras.regularizers.l2(self.regul))

    super().build(input_shape)        # Appel de la méthode build()

  # Définit la logique de la couche d'attention
  # Arguments :     x : (batch_size, Nbr_Sequence, taille_fenetre)
  #                 y : (batch_size, taille_fenetre)
  # Exemple :   batch_size = 32
  #             Nbr_Sequence =30
  #             taille_fenetre = 20
  #             dim_GRU = 40 
  def call(self,x,y):
    # Création des vecteurs mi dans le tenseur M
    M = tf.expand_dims(x,axis=-1)                                   # (32,30,20) => (32,30,20,1)
    M = tf.keras.layers.TimeDistributed(self.couche_GRU_A)(M)       # (32,30,20,1) => (32,30,40) : TimeStep = 30 : (32,20,1) envoyé
    M = K.tanh(M)

    # Création du vecteur d'état u
    u = tf.expand_dims(y,axis=-1)                                   # (32,20) => (32,20,1)
    u = self.couche_GRU_B(u)                                        # (32,20,1) => (32,40)
    u = tf.expand_dims(u,axis=-1)                                   # (32,40) => (32,40,1)
    u = K.tanh(u)                                                   # (32,40,1)

    # Calcul des poids d'attention
    p = tf.matmul(M,u)                                              # (32,30,40)x(32,40,1)=(32,30,1)
    p = tf.keras.activations.softmax(p,axis=1)                      # (32,30,1)

    # Création des vecteurs ci dans le tenseur C
    C = tf.expand_dims(x,axis=-1)                                   # (32,30,20) => (32,30,20,1)
    C = tf.keras.layers.TimeDistributed(self.couche_GRU_C)(C)       # (32,30,20,1) => (32,30,40) : TimeStep = 30 : (32,20,1) envoyé
    C = K.tanh(C)

    # Calcul du vecteur réponse issu de la mémoire
    o = tf.multiply(C,p)                                            # (32,30,40)_x_(32,30,1) = (32,30,40)
    o = K.sum(o, axis=1)                                            # (32,40)
    o = K.tanh(o)                                                   # (32,40)
    
    # Retourne le vecteur d'attention
    return (o+tf.squeeze(u,axis=2))


In [ ]:
dim_LSTM = 40
l1_reg = 0.0
l2_reg = 0.0

# Définition des entrées du modèle
entrees_sequences = tf.keras.layers.Input(shape=(Nbr_Sequences,taille_fenetre))
entrees_entrainement = tf.keras.layers.Input(shape=(taille_fenetre))

# Encodeur
s_encodeur = Couche_End_to_End_MN(dim_GRU=dim_LSTM,regul=0.0)(entrees_sequences,entrees_entrainement)

# Décodeur
s_decodeur = tf.keras.layers.Dense(dim_LSTM,activation="tanh")(s_encodeur)
s_decodeur = tf.keras.layers.Concatenate()([s_decodeur,s_encodeur])

# Générateur
sortie = tf.keras.layers.Dense(1)(s_decodeur)

# Construction du modèle
model = tf.keras.Model([entrees_sequences,entrees_entrainement],sortie)
model.summary()

**2. Optimisation de l'apprentissage**

Pour accélérer le traitement des données, nous n'allons pas utiliser l'intégralité des données pendant la mise à jour du gradient, comme cela a été fait jusqu'à présent (en utilisant le dataset).  
Cette fois-ci, nous allons forcer les mises à jour du gradient à se produire de manière moins fréquente en attribuant la valeur du batch_size à prendre en compte lors de la regression du modèle.  
Pour cela, on utilise l'argument "batch_size" dans la méthode fit. En précisant un batch_size=1000, cela signifie que :
 - Sur notre total de 56000 échantillons, 56 seront utilisés pour les calculs du gradient
 - Il y aura également 56 itérations à chaque période.
  
    
    
Si nous avions pris le dataset comme entrée, nous aurions eu :
- Un total de 56000 échantillons également
- Chaque période aurait également pris 56 itérations pour se compléter
- Mais 1000 échantillons auraient été utilisés pour le calcul du gradient, au lieu de 56 avec la méthode utilisée.

In [ ]:
batch_size = 1000

In [ ]:
# Définition de la fonction de régulation du taux d'apprentissage
def RegulationTauxApprentissage(periode, taux):
  return 1e-8*10**(periode/10)

# Définition de l'optimiseur à utiliser
optimiseur=tf.keras.optimizers.Adam()

# Compile le modèle
model.compile(loss="logcosh", optimizer=optimiseur, metrics="mse")

# Utilisation de la méthode ModelCheckPoint
CheckPoint = tf.keras.callbacks.ModelCheckpoint("poids.hdf5", monitor='loss', verbose=1, save_best_only=True, save_weights_only = True, mode='auto', save_freq='epoch')

# Entraine le modèle en utilisant notre fonction personnelle de régulation du taux d'apprentissage
historique = model.fit(dataset,epochs=100,verbose=1, callbacks=[tf.keras.callbacks.LearningRateScheduler(RegulationTauxApprentissage), CheckPoint],batch_size=batch_size)

In [ ]:
# Construit un vecteur avec les valeurs du taux d'apprentissage à chaque période 
taux = 1e-8*(10**(np.arange(100)/10))

# Affiche l'erreur en fonction du taux d'apprentissage
plt.figure(figsize=(10, 6))
plt.semilogx(taux,historique.history["loss"])
plt.axis([ taux[30], taux[99], 0, 0.04])
plt.title("Evolution de l'erreur en fonction du taux d'apprentissage")

In [ ]:
# Chargement des poids sauvegardés
model.load_weights("poids.hdf5")

In [ ]:
max_periodes = 1000

# Classe permettant d'arrêter l'entrainement si la variation
# devient plus petite qu'une valeur à choisir sur un nombre
# de périodes à choisir
class StopTrain(keras.callbacks.Callback):
    def __init__(self, delta=0.01,periodes=100, term="loss", logs={}):
      self.n_periodes = 0
      self.periodes = periodes
      self.loss_1 = 100
      self.delta = delta
      self.term = term
    def on_epoch_end(self, epoch, logs={}):
      diff_loss = abs(self.loss_1 - logs[self.term])
      self.loss_1 = logs[self.term]
      if (diff_loss < self.delta):
        self.n_periodes = self.n_periodes + 1
      else:
        self.n_periodes = 0
      if (self.n_periodes == self.periodes):
        print("Arrêt de l'entrainement...")
        self.model.stop_training = True

def  My_MSE(y_true,y_pred):
  return(tf.keras.metrics.mse(y_true,y_pred)*std.numpy()+mean.numpy())
  
# Définition des paramètres liés à l'évolution du taux d'apprentissage
lr_schedule = tf.keras.optimizers.schedules.InverseTimeDecay(
    initial_learning_rate=0.001,
    decay_steps=10,
    decay_rate=0.01)

# Définition de l'optimiseur à utiliser
optimiseur=tf.keras.optimizers.Adam(learning_rate=lr_schedule)


# Utilisation de la méthode ModelCheckPoint
CheckPoint = tf.keras.callbacks.ModelCheckpoint("poids_train.hdf5", monitor='loss', verbose=1, save_best_only=True, save_weights_only = True, mode='auto', save_freq='epoch')

# Compile le modèle
model.compile(loss="logcosh", optimizer=optimiseur, metrics=["mse",My_MSE])

# Entraine le modèle, avec une réduction des calculs du gradient
#historique = model.fit(x=x_train,y=y_train,validation_data=(x_val,y_val), epochs=max_periodes,verbose=1, callbacks=[CheckPoint,StopTrain(delta=0.005,periodes = 10, term="My_MSE")],batch_size=batch_size)

# Entraine le modèle sans réduction de calculs
historique = model.fit(dataset,validation_data=dataset_val, epochs=max_periodes,verbose=1, callbacks=[CheckPoint,StopTrain(delta=0.1,periodes = 10, term="val_My_MSE")])


In [ ]:
from google.colab import files
files.download('poids_train.hdf5')

In [ ]:
erreur_entrainement = historique.history["loss"]
erreur_validation = historique.history["val_loss"]

# Affiche l'erreur en fonction de la période
plt.figure(figsize=(10, 6))
plt.plot(np.arange(0,len(erreur_entrainement)),erreur_entrainement, label="Erreurs sur les entrainements")
plt.plot(np.arange(0,len(erreur_entrainement)),erreur_validation, label ="Erreurs sur les validations")
plt.legend()

plt.title("Evolution de l'erreur en fonction de la période")

In [ ]:
# Evaluation du modèle
# Avec le modèle type encodeur/décodeur :
# 56/56 [==============================] - 7s 113ms/step - loss: 1.1795e-04 - mse: 2.3664e-04 - My_MSE: 2712.2415
# 14/14 [==============================] - 2s 113ms/step - loss: 0.3602 - mse: 1.9360 - My_MSE: 9650.2637
model.evaluate(dataset)
model.evaluate(dataset_val)


**3. Prédictions**

In [ ]:
horizon = 1
# Création des instants d'entrainement et de validation
y_train_timing = serie_entrainement.index[taille_fenetre + horizon - 1:taille_fenetre + horizon - 1+len(y_train)]
y_val_timing = serie_test.index[taille_fenetre + horizon - 1:taille_fenetre + horizon - 1+len(y_val)]

# Calcul des prédictions
pred_ent = model.predict(x_train, verbose=1)
pred_val = model.predict(x_val, verbose=1)

**4.Affichage sur une période de 1 heure**

Création d'une série contenant les valeurs originales et les prédictions, synchronisées dans le temps :

In [ ]:
serie_btc_ent_ori = serie_entrainement*std+mean
serie_btc_val_ori = serie_test*std+mean

serie_btc_ent_pred = pd.Series(data=(pred_ent[:,0]*std+mean),index=y_train_timing)
serie_btc_val_pred = pd.Series(data=(pred_val[:,0]*std+mean),index=y_val_timing)

serie_btc_ori = pd.concat([serie_btc_ent_ori,serie_btc_val_ori])
serie_btc_pred = pd.concat([serie_btc_ent_pred,serie_btc_val_pred])

serie_btc_ori = serie_btc_ori.fillna(method="backfill")
serie_btc_pred = serie_btc_pred.fillna(method="backfill")

serie_btc_ent_ori = serie_btc_ent_ori.fillna(method="backfill")
serie_btc_val_ori = serie_btc_val_ori.fillna(method="backfill")
serie_btc_ent_pred = serie_btc_ent_pred.fillna(method="backfill")
serie_btc_val_pred = serie_btc_val_pred.fillna(method="backfill")

frame = {'BTC_ALL' : serie_btc_ori, 'BTC_ENT': serie_btc_ent_ori, 'BTC_VAL' : serie_btc_val_ori, 'BTC_PRED' : serie_btc_pred, 'BTC_PRED_ENT' : serie_btc_ent_pred, 'BTC_PRED_VAL':serie_btc_val_pred}
df_resultats = pd.DataFrame(frame)
df_resultats

In [ ]:
date_separation

In [ ]:
df_resultats.loc[date_separation-pd.Timedelta(hours=7):date_separation+pd.Timedelta(hours=7)]

In [ ]:
import plotly.graph_objects as go

fig = go.Figure()

# Courbe originale
fig.add_trace(go.Scatter(x=df_resultats.index,y=df_resultats['BTC_ALL'],line=dict(color='blue', width=1),name="Prix BTC"))

# Courbes des prédictions d'entrainement
fig.add_trace(go.Scatter(x=df_resultats.index,y=df_resultats['BTC_PRED_ENT'],line=dict(color='green', width=1),name="Entrainement"))

# Courbe de validation
fig.add_trace(go.Scatter(x=df_resultats.index,y=df_resultats['BTC_PRED_VAL'],line=dict(color='red', width=1),name="Validation"))

fig.update_xaxes(rangeslider_visible=True)
yaxis=dict(autorange = True,fixedrange= False)
fig.update_yaxes(yaxis)
fig.show()

**5. Détection de l'augmentation des erreurs dans la zone de validation**

On peut imaginer devoir suivre en temps réel l'évolution des prédictions afin de détecter lorsque le modèle n'est plus valide.

In [ ]:
# Détection de la date où commencent les anomalies
# sur la période de validation

# Construit les dataframe pour calculer les erreurs
df_error_btc_ent = df_resultats[['BTC_ENT','BTC_PRED_ENT']].dropna()
df_error_btc_val = df_resultats[['BTC_VAL','BTC_PRED_VAL']].dropna()

# Calcul des erreurs relatives sur les entrainements et les validations
erreur_ent = abs(np.asarray(df_error_btc_ent['BTC_ENT']) - np.asarray(df_error_btc_ent['BTC_PRED_ENT']))/np.asarray(df_error_btc_ent['BTC_ENT'])*100.0
erreur_val = abs(np.asarray(df_error_btc_val['BTC_VAL']) - np.asarray(df_error_btc_val['BTC_PRED_VAL']))/np.asarray(df_error_btc_val['BTC_VAL'])*100.0

# Erreur relative moyenne sur les entrainements
erreur_ent_mape = tf.keras.metrics.mape(np.asarray(df_error_btc_ent['BTC_ENT']),np.asarray(df_error_btc_ent['BTC_PRED_ENT']))
erreur_ent_mape_max = np.amax(erreur_ent)

# Calcul le nombre d'anomalies sur les entrainements avec le ratio spécifié
nbr_anomalies_ent = np.count_nonzero(erreur_ent>erreur_ent_mape)

# Calcul du ratio d'anomalies sur la période d'entrainement
ratio_ent = np.count_nonzero(erreur_ent>erreur_ent_mape)/erreur_ent.size

# Recherche de la date à partir de laquelle on dépasse l
n_erreurs = 0
for i in range(5,erreur_val.size):
  erreur_val_ = abs(np.asarray(df_error_btc_val['BTC_VAL'][i]) - np.asarray(df_error_btc_val['BTC_PRED_VAL'][i]))/np.asarray(df_error_btc_val['BTC_VAL'][i])*100.0
  seuil = erreur_ent_mape_max*1
  if erreur_val_ > seuil:
    n_erreurs = n_erreurs + 1
    if n_erreurs == 5:
      index = df_error_btc_val.index[i]
      break
  else:
    n_erreurs = 0
print(index)

In [ ]:
import plotly.graph_objects as go

fig = go.Figure()

# Courbe originale
fig.add_trace(go.Scatter(x=df_resultats.index,y=df_resultats['BTC_ALL'],line=dict(color='blue', width=1),name="Prix BTC"))

# Courbes des prédictions d'entrainement
fig.add_trace(go.Scatter(x=df_resultats.index,y=df_resultats['BTC_PRED_ENT'],line=dict(color='green', width=1),name="Entrainement"))

# Courbe de validation
fig.add_trace(go.Scatter(x=df_resultats.index,y=df_resultats['BTC_PRED_VAL'],line=dict(color='red', width=1),name="Validation"))

# Délimite la zone d'erreur
fig.add_shape(type="line",x0=index,y0=0,x1=index,y1=np.amax(df_resultats['BTC_ALL']),name="Zone d'erreur")

# Affiche les courbes
fig.update_xaxes(rangeslider_visible=True)
yaxis=dict(autorange = True,fixedrange= False)
fig.update_yaxes(yaxis)
fig.show()

**5.Affichage sur une période de 1 jour**

In [ ]:
# Echantillonage des données
time = "1D"

df_resultats = df_resultats.resample(time).asfreq()
df_resultats

In [ ]:
import plotly.graph_objects as go

fig = go.Figure()

# Courbe originale
fig.add_trace(go.Scatter(x=df_resultats.index,y=df_resultats['BTC_ALL'],line=dict(color='blue', width=1),name="Prix BTC"))

# Courbes des prédictions d'entrainement
fig.add_trace(go.Scatter(x=df_resultats.index,y=df_resultats['BTC_PRED_ENT'],line=dict(color='green', width=1),name="Entrainement"))

# Courbe de validation
fig.add_trace(go.Scatter(x=df_resultats.index,y=df_resultats['BTC_PRED_VAL'],line=dict(color='red', width=1),name="Validation"))

# Délimite la zone d'erreur
fig.add_shape(type="line",x0=index,y0=0,x1=index,y1=np.amax(df_resultats['BTC_ALL']),name="Zone d'erreur")

# Affiche les courbes
fig.update_xaxes(rangeslider_visible=True)
yaxis=dict(autorange = True,fixedrange= False)
fig.update_yaxes(yaxis)
fig.show()

**6.Synthèse des erreurs sur les différentes zones**

In [ ]:
# Echantillonage des données
time = "1H"

df_resultats = df_resultats.resample(time).asfreq()
df_resultats

In [ ]:
# Détection de la date où commencnte les anomalies
# sur la période de validation

# Construit les dataframe pour calculer les erreurs
df_error_btc_ent = df_resultats[['BTC_ENT','BTC_PRED_ENT']].dropna()
df_error_btc_val = df_resultats[['BTC_VAL','BTC_PRED_VAL']].dropna()


In [ ]:
# Erreurs d'entrainement
mae_ent = tf.keras.metrics.mean_absolute_error(np.asarray(df_error_btc_ent['BTC_ENT']),np.asarray(df_error_btc_ent['BTC_PRED_ENT'])).numpy()
mse_ent = tf.keras.metrics.mean_squared_error(np.asarray(df_error_btc_ent['BTC_ENT']),np.asarray(df_error_btc_ent['BTC_PRED_ENT'])).numpy()
mape_ent = tf.keras.metrics.mape(np.asarray(df_error_btc_ent['BTC_ENT']),np.asarray(df_error_btc_ent['BTC_PRED_ENT'])).numpy()

# Erreurs de validation
mae_val = tf.keras.metrics.mean_absolute_error(np.asarray(df_error_btc_val['BTC_VAL']),np.asarray(df_error_btc_val['BTC_PRED_VAL'])).numpy()
mse_val = tf.keras.metrics.mean_squared_error(np.asarray(df_error_btc_val['BTC_VAL']),np.asarray(df_error_btc_val['BTC_PRED_VAL'])).numpy()
mape_val = tf.keras.metrics.mape(np.asarray(df_error_btc_val['BTC_VAL']),np.asarray(df_error_btc_val['BTC_PRED_VAL'])).numpy()

# Erreurs sur la zone valide de validation
mae_val_ok = tf.keras.metrics.mean_absolute_error(df_error_btc_val['BTC_VAL'][:index],np.asarray(df_error_btc_val['BTC_PRED_VAL'][:index])).numpy()
mse_val_ok = tf.keras.metrics.mean_squared_error(df_error_btc_val['BTC_VAL'][:index],np.asarray(df_error_btc_val['BTC_PRED_VAL'][:index])).numpy()
mape_val_ok = tf.keras.metrics.mape(df_error_btc_val['BTC_VAL'][:index],np.asarray(df_error_btc_val['BTC_PRED_VAL'][:index])).numpy()


print("Erreur mae entrainement %s" %mae_ent)
print("Erreur mse entrainement : %s" %mse_ent)
print("Erreur mape entrainement : %s\n" %mape_ent)

print("Erreur mae validation %s" %mae_val)
print("Erreur mse validation : %s" %mse_val)
print("Erreur mape entrainement : %s\n" %mape_val)

print("Erreur mae validation zone OK %s" %mae_val_ok)
print("Erreur mse validation zone OK: %s" %mse_val_ok)
print("Erreur mape validation zone OK: %s" %mape_val_ok)